In [ ]:
pip install fireducks

In [ ]:
!pip install memory-profiler


In [ ]:
pip install pandasql

### Importing packages and setting work directory


In [ ]:
pip install pandasql

In [2]:
import pandas as pd
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import time
from pandasql import sqldf
os.chdir(r'C:\Users\Tamir\OneDrive - Technion\current_work_13_08\service systems')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

## Sorting by agent (id_rep), time and deleting all id_rep = 1 in big dataframe (session event)

In [3]:
session_events_1 = pd.read_csv('merged_session_events.csv')
columns_to_sort = [session_events_1.columns[3], session_events_1.columns[5]]
#test_df = session_events_1.sort_values(by=columns_to_sort, ascending=[True, True])
#test_df = test_df[test_df[' id_rep'] != 1]

In [4]:
temp_s = session_events_1.groupby(" id_session").size().reset_index(name = "count")

temp_s[temp_s["count"] == 1]

,id_session,count
2526,100002573,1
2711,100002761,1
4258,100004332,1
4314,100004390,1
4382,100004458,1
...,...,...
333854,100362697,1
333855,100362698,1
333856,100362699,1
333857,100362700,1


In [ ]:
session_events_1.columns

In [ ]:
temp_t = session_events_1[session_events_1[" id_rep"] != 1]

temp_t[(temp_t[" event_type"] == 1) | (temp_t[" event_type"] == 2)]

In [ ]:
session_events_1[session_events_1[" event_type_desc"] == ' rep_line'][" sentiment"].value_counts()


In [ ]:
session_events_1[session_events_1[" event_type_desc"] == 'visitor_line'][" sentiment"]


In [13]:
session_events_merged = pd.read_csv('raw_session_events_before_events_8_9_19_07.csv')

C:\Users\Tamir\AppData\Local\Temp\ipykernel_29328\3828195404.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  session_events_merged = pd.read_csv('raw_session_events_before_events_8_9_19_07.csv')


In [14]:
session_events_merged.columns

Index(['id_site', ' id_session', ' id_visitor', ' start_date', ' start_time',
       ' end_date', ' end_time', ' duration', ' number_words', ' number_chars',
       ' number_lines', ' answer_canned', ' sentiment', ' accept_date',
       ' accept_time', ' read_date', ' read_time', ' event_type',
       ' event_type_desc', ' outcome', ' outcome_desc', ' subsession',
       ' delay', ' id_rep_code', ' sentiment_type', ' id_agent',
       ' id_agent_code', 'source_file', 'queue_exit_date', 'assignment_date',
       'event_id', 'first_msg_dummy', ' id_rep'],
      dtype='object')

In [8]:
session_events_merged['assignment_date'] = session_events_merged['assignment_date'].astype(str)

In [ ]:
session_events_merged.to_csv('raw_session_events_before_events_8_9_19_07.csv', index= 0)

## Setting t_rows with the appropripiate columns for simplicity:

In [9]:
t_rows_split = session_events_merged[['event_id', " id_visitor", ' id_session', " id_rep", " start_date", " start_time"," end_date",
                       " end_time"," duration", " event_type", " event_type_desc", ' sentiment', ' number_words', 'assignment_date', 'first_msg_dummy']].reset_index(drop=True)
# We could slice here to decrease loading time in the big processing code.

## Deleting 8 and 9 from evet_type columns:

In [10]:
# These variables seems not relevant since they regard to agents handling parallel chats and the time the customer waits in these occasions.
dele = t_rows_split[(t_rows_split[' event_type'] == 8) | (t_rows_split[' event_type'] == 9)].index
t_rows_split.drop(dele , inplace=True)
# Sorting again the data:
df_temp = t_rows_split.sort_values(by=[' id_session', ' end_time'])
df_temp =  df_temp.reset_index(drop=True)
df_temp.shape

(4964895, 15)

Criterions for unions:

1. same event type
2. same id_session
3. 




In [ ]:
df_temp.sort_values(by = [' id_rep', ' end_time'], inplace= True)
df_temp =  df_temp.reset_index(drop=True)
df_temp[[' id_session', ' event_type_desc', ' end_date', ' id_rep']].head(100)


In [ ]:
df_temp[df_temp['first_msg_dummy'] == 1].shape

In [ ]:
df_temp.shape

In [ ]:
df_temp_1 = df_temp.iloc[:1000]
df_temp_2 = df_temp.iloc[1000000:2000000]
df_temp_3= df_temp.iloc[2000000:3000000]
df_temp_4= df_temp.iloc[3000000:4000000]
df_temp_5= df_temp.iloc[4000000:]

In [ ]:
df_temp_1

In [ ]:
df_temp_1.to_csv('df_temp_for_gpt.csv', index = 0)

## Handling same and consecutive event types (event type 1 & 2):

##### Setting the desired data structure in order to sum some of the variables and keep the the others from the first row (from the same consecutive rows group). 

##### We implemented a boolean condition that assigns TRUE to rows that are different than the next one by EITHER event type or id session.
##### Relying on this condition, we used cumsum and groupby to create the groups with only the SAME id session & event type (FALSE in the condition), cumsum was activated only when condition showed TRUE, therefore, it differentiated between groups only when desired.

##### Finally grouped is a groupby type which is further processed according to our desired outcomes (values are being summed, or kept as the first row).

In [ ]:
column_to_sum_by = ' event_type'
# Columns to sum
columns_to_sum = [' sentiment', ' duration', ' number_words']
# Columns to to keep original val
columns_to_keep_original_val = ['event_id', ' id_visitor',' id_session', ' id_rep', ' start_date', ' start_time',
                               ' end_date', ' end_time', ' event_type', ' event_type_desc', 'first_msg_dummy', 'assignment_date']

# Define the condition for merging based on 'session_id' and 'event_type'
condition = (df_temp_1[column_to_sum_by] != df_temp_1[column_to_sum_by].shift()) | (df_temp_1[' id_session'] != df_temp_1[' id_session'].shift())

# Group consecutive rows based on the defined condition
grouped = df_temp_1.groupby(condition.cumsum())
print(grouped)
# Define a function to merge consecutive rows and sum values at specific columns

##### Creating a function that first ignores groups that contain only 1 row, and then sums the columns to sum (sentiment, duration, number of words), and then keeps the relevant columns of the first row in the group.

##### It creates different dictionaries for each action, and then merges the dictionaries into one, and then returns it as a dataframe. The function is applied on the 'grouped' groupby type.

In [ ]:
# Start timing
start_time = time.time()

def merge_consecutive_rows(group):
    if len(group) == 1:
        return group
    else:
        # Sum values at specific columns
        summed_values = group[columns_to_sum].sum()
        # Keep the value of the first row for other columns
        first_row_values = group.iloc[0][columns_to_keep_original_val]
        # Create a new row with first_row_values followed by summed_values
        new_row = {**first_row_values.to_dict(), **summed_values.to_dict()}
        return pd.DataFrame([new_row], columns=group.columns)

# Apply the function to each group and concatenate the results
merged_df = grouped.apply(merge_consecutive_rows).reset_index(drop=True)
merged_df.shape

# End timing
end_time = time.time()

# Print execution time
print(f"Execution time: {end_time - start_time:.2f} seconds")

In [ ]:
# Start timing
start_time = time.time()

def merge_consecutive_rows(group):
    group_size = len(group)

    if group_size == 1:
        # Keep the original row, append num_events=1
        return group.assign(num_events=1)

    # Sum values at specific columns
    summed_values = group[columns_to_sum].sum()

    # Keep the value of the first row for other columns
    first_row_values = group.iloc[0][columns_to_keep_original_val]

    # Build new row dict and add the count
    new_row = {
        **first_row_values.to_dict(),
        **summed_values.to_dict(),
        "num_events": group_size
    }

    # Return as a one-row DataFrame (let pandas infer column order)
    return pd.DataFrame([new_row])

# Apply the function to each group and concatenate the results
merged_df = grouped.apply(merge_consecutive_rows).reset_index(drop=True)

# (Optional) ensure num_events is the last column
if "num_events" in merged_df.columns:
    cols = [c for c in merged_df.columns if c != "num_events"] + ["num_events"]
    merged_df = merged_df[cols]

merged_df.shape

# End timing
end_time = time.time()

# Print execution time
print(f"Execution time: {end_time - start_time:.2f} seconds")

In [ ]:
df_temp_1[[' id_rep', ' id_session', ' event_type_desc', ' number_words', ' end_date']].head(100)

In [ ]:
merged_df[[' id_rep', ' id_session', ' event_type_desc', ' number_words', ' end_date', 'num_events']].head(100)

In [ ]:
#saving to csv: 
merged_df.to_csv('merged_df_after_dan_6m_renewal_04_10_example.csv', index = 0)

In [ ]:
merged_df_1 = pd.read_csv('merged_df_after_dan_6m_renewal_16_08_part_1.csv')
merged_df_2 = pd.read_csv('merged_df_after_dan_6m_renewal_16_08_part_2.csv')
merged_df_3 = pd.read_csv('merged_df_after_dan_6m_renewal_16_08_part_3.csv')
merged_df_4 = pd.read_csv('merged_df_after_dan_6m_renewal_16_08_part_4.csv')
merged_df_5 = pd.read_csv('merged_df_after_dan_6m_renewal_16_08_part_5.csv')

In [ ]:
merged_df = pd.concat([merged_df_1, merged_df_2, merged_df_3, merged_df_4, merged_df_5])

In [ ]:
merged_df.to_csv('merged_df_after_dan_6m_renewal_16_08_FULL.csv', index= 0)

In [ ]:
merged_df = pd.read_csv('merged_df_after_dan_6m_renewal_16_08_FULL.csv')

In [ ]:
merged_df.shape

In [ ]:
merged_df.head(50)

In [ ]:
merged_df[' id_session'].nunique()

In [ ]:
#sort by time:
merged_df = merged_df.sort_values(by=[' id_rep', ' end_time' ])
merged_df = merged_df.reset_index(drop=True)
merged_df.head()

In [ ]:
merged_df[merged_df[' id_session'] == 100000002]

In [ ]:
merged_df[merged_df['event_id'].duplicated()].shape

## Creating concurrencies

### Looking at merged_session, as session level

In [ ]:
#If you have filtered merged just read it to here using: 
merged_sessions = pd.read_csv('merged_session.csv')

#### if youve sliced merged_df (i.e event level df) differently, you need to extract all the unique id sessions that are in the table. This will extract all the unique sessions to a list.

In [ ]:
def uniqe(merged_df):
    uniqe = merged_df[' id_session'].unique()
    #Reading the merged sessions dataframe:
    merged_session = pd.read_csv('merged_session.csv')
    #Filtering all the intrested sessions from the merged sessions dataframe:
    filter1 = merged_session[' id_session'].isin(uniqe)
    merged_sessions = merged_session[filter1]
    #Turning it to a csv:
    #merged_sessions.to_csv('merged_sessions.csv')
    return merged_sessions
    
merged_sessions = uniqe(merged_df).reset_index(drop = True)
merged_sessions.shape

#### The next part requried alot of loading power in order to create a documentation of concurrencies, and evantually creating choice sets. Therefore, we attempt to extract the relevant columns from the session level df into the event level df, in order for the algorithm to focus only on one df. 

#### These are the columns we want from merged_sessions:

In [ ]:
columns_to_keep = [' chat_end_time', ' chat_start_time', ' id_session', ' chat_start_date', ' chat_end_date']

df_columns_to_keep = merged_sessions[columns_to_keep]

# Merge on id_session and filter for time range
df_merged = merged_df.merge(df_columns_to_keep, on=' id_session', how='left')

# Keep only rows where end_time (left) is within the start_time and end_time (right)
df_filtered = df_merged[(df_merged[' end_time'] >= df_merged[' chat_start_time']) & 
                        (df_merged[' end_time'] <= df_merged[' chat_end_time'])]

df_filtered.shape

In [ ]:
merged_df[' id_session'].nunique()

In [ ]:
merged_sessions[' id_session'].nunique()

#### 23.3 fixing the merge between the session level and event level dfs, we were about to delete duplicated events after the merge and check for descriptives of the columns in order to see if theres any difference.

In [ ]:
df_dup_events = df_filtered[df_filtered['event_id'].duplicated()]

df_dup_events

##### Deleting all duplicated columns after merge and filtering.

In [ ]:
df_no_dupes = df_filtered[~df_filtered['event_id'].isin(df_dup_events['event_id'])]

print(df_no_dupes.shape, df_filtered.shape)

In [ ]:
#Inserting all messages sent by the agent to a dataframe:
filtered_rows = df_no_dupes[df_no_dupes[' event_type'] == 2]

# Start timing for loading time of the code:
start_time = time.time()

# this function indetifies concurrent sessions
def get_data_for_an_identifier(row):
    print('\r', row['index'], end='')
    # filtering the session level dataframe according to the agent number that is shown in filtered_rows (the filtered df with only the agent msg)
    # tmp_df is a session level dataframe that shows details of all the sessions that the given agent is a part of.
    tmp_df = merged_sessions[merged_sessions[' id_rep'] == row[' id_rep']]
    # identifying the sessions that started before the end_time of the agent's message in filtered_rows, 
    # and ended after the end_time of the agent's message.
    tmp_df = tmp_df[tmp_df[' chat_end_time'] > row[' end_time']]
    tmp_df = tmp_df[tmp_df[' chat_start_time'] < row[' end_time']]
    # returning the time of msg, chosen session, and all the concurrent sessions in a list.
    return {'id_rep': row[' id_rep'], 'event_id': row['event_id'], 'time':[row[' end_time']],
            'session_id_chosen':[row[' id_session']], 'concurrent_sessions':tmp_df[' id_session'].tolist()}

# applying the function on filtered_rows
result_dicts  = filtered_rows.reset_index().apply(lambda row: get_data_for_an_identifier(row), axis=1)
# creating df with returned dictionary
aggregated_df = pd.DataFrame(list(result_dicts))
aggregated_df.head(50)

# End timing
end_time = time.time()

# Print execution time
print(f"Execution time: {end_time - start_time:.2f} seconds")

In [ ]:
#SAVING TO CSV:
aggregated_df.to_csv('before_second_stage_all_data_23_08.csv', index = 0)

In [ ]:
aggregated_df = pd.read_csv('before_second_stage_all_data_23_08.csv')

In [ ]:
aggregated_df

Creating Workload variable, i.e number of concurrent sessions the agent had

In [ ]:
aggregated_df['workload'] = aggregated_df['concurrent_sessions'].apply(lambda x: len(x))

As the concurrent sessions are in lists for each message by the agent, we want to create distinct rows for each concurrent session that is concurrent to the message by the agent. Therefore, we use pandas 'explode' to do so

In [ ]:
from ast import literal_eval

# We run literal_eval on these 3 columns to transform the strings that are shown as lists in the rows to actual lists
aggregated_df['concurrent_sessions'] = aggregated_df['concurrent_sessions'].apply(literal_eval)
aggregated_df['time'] = aggregated_df['time'].apply(literal_eval)
aggregated_df['session_id_chosen'] = aggregated_df['session_id_chosen'].apply(literal_eval)

# filtering empty concurrent sessions
filtered_agg = aggregated_df[aggregated_df['concurrent_sessions'].apply(lambda x: len(x) > 1)]
filtered_agg = filtered_agg.reset_index(drop = True)

#Exploding the dataframe (each session in the list is becoming a new row):
df_exploded = filtered_agg.explode('concurrent_sessions')
df_exploded = df_exploded.explode('time')
df_exploded = df_exploded.explode('session_id_chosen')
#Reseting index but saving the index column for choiceset:
df_exploded = df_exploded.reset_index()
df_exploded.head()

In [ ]:
filtered_agg.shape[0]

In [ ]:
df_exploded.drop(columns= ['index', 'Unnamed: 0'], inplace= True)

In [ ]:
def chosen_tracker(df):
    df['chosen'] = (df['session_id_chosen'] == df['concurrent_sessions']).astype(int)
    return df
df_exploded = chosen_tracker(df_exploded)
df_exploded.head()

In [ ]:
df_exploded.to_csv('df_exploded_all_data_24_08.csv', index = 0)

In [ ]:
df_exploded = pd.read_csv('df_exploded_all_data_24_08.csv')

In [ ]:
df_exploded.shape

In [ ]:
df_exploded_sliced = df_exploded.iloc[:500000]

In [ ]:
df_exploded_1 = pd.read_csv('df_exploded_chosen_0to1m.csv', index_col = 0)
df_exploded_2 = pd.read_csv('df_exploded_chosen_2m.csv', index_col = 0)

In [ ]:
df_exploded_1

In [ ]:
df_exploded_2

In [ ]:
df_exploded_1.drop(columns={'index'}, inplace= True)
df_exploded_2.drop(columns={'index'}, inplace= True)

In [ ]:
df_exploded_1.shape

In [ ]:
df_exploded_2.shape

In [ ]:
df_exploded = pd.concat([df_exploded_1, df_exploded_2])
df_exploded.reset_index(drop = True, inplace = True)

df_exploded.shape

In [ ]:
df_exploded.to_csv('df_exploded_0_to_2m_18_06.csv', index = 0)

In [ ]:
import pandas as pd
import time
import warnings

warnings.filterwarnings('ignore')

# Start timing
start_time = time.time()

# STEP 1: Preprocess merged_df
# Make sure to strip column names (remove extra spaces)
merged_df.columns = merged_df.columns.str.strip()
df_exploded_sliced.columns = df_exploded_sliced.columns.str.strip()

# Sort for efficient filtering
merged_df = merged_df.sort_values(['id_session', 'end_time'])

# Group by session for fast access
session_groups = dict(tuple(merged_df.groupby('id_session')))

# List to collect results
filtered_events_list = []

# STEP 2: Iterate over df_exploded_sliced
for row in df_exploded_sliced.reset_index().itertuples(index=False):
    session_events = session_groups.get(row.concurrent_sessions, pd.DataFrame())

    if session_events.empty:
        continue

    # Get only events before the choice time
    past_events = session_events[session_events['end_time'] < row.time]

    if past_events.empty:
        continue

    # Get the latest event
    closest_time = past_events['end_time'].max()
    filtered_event = past_events[past_events['end_time'] == closest_time]

    # Keep only event_type == 1
    filtered_event = filtered_event[filtered_event['event_type'] == 1]

    if not filtered_event.empty:
        # Enrich the row
        filtered_event = filtered_event.assign(
            choice_set=row.index,
            chosen=row.chosen,
            waiting_time=row.time - filtered_event['end_time'],
            workload=row.workload
        )

        filtered_events_list.append(filtered_event)

# STEP 3: Combine and save
if filtered_events_list:
    all_filtered_events = pd.concat(filtered_events_list, ignore_index=True)
    all_filtered_events.to_csv('all_filtered_events_3m_f.csv', index=False)
    print(all_filtered_events.head())
else:
    print("No filtered events found.")

# STEP 4: Execution time
end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")


In [ ]:
all_filtered_events.iloc[10000:10500]

In [ ]:
# Initialize an empty list to store all df_t_filtered DataFrames
all_filtered_dfs = pd.DataFrame()
##### Function to get max event
def max_event(row):
    print('\r', row['index'], end='')
    global all_filtered_dfs
    #Filtering merged df based on the session in row(exploded df)
    df_t = merged_df[merged_df[' id_session'] == row['concurrent_sessions']]
    #Filtering df_t (filtered merged_df) that only events that happend earlier than the choice time in row(exploded_df)
    filtered_df = df_t[df_t[' end_time'] < row['time']]
    filtered_df['index'] = row['index']
    if not filtered_df.empty:# enter this condition only if the dataframe is not empty, maybe this will save some time
        #Take the max number out of the filtered_df
        closest_time = filtered_df[' end_time'].max()
        df_t_filtered = filtered_df[filtered_df[' end_time'] == closest_time]
        # Filter based on event_type
        df_t_filtered = df_t_filtered[df_t_filtered[' event_type'] == 1]
        #Assign cohice_set number based on index column in row(exploded_df)
        df_t_filtered['choice_set'] = row['index'] 
        df_t_filtered['chosen'] = row['chosen']
        all_filtered_dfs = pd.concat([all_filtered_dfs, df_t_filtered])
  # Append df_t_filtered to the list 
        return all_filtered_dfs

# Apply the max_event function to each row of df_exploded
result_dicts  = filtered_rows.reset_index().apply(lambda row: get_data_for_an_identifier(row), axis=1)
result = df_exploded.apply(max_event, axis=1)

In [ ]:
#Initialize an empty list to store all df_t_filtered DataFrames
all_filtered_dfs = []
##### Function to get max event
def max_event(row):
    print('\r', row['index'], end='')
    #Filtering merged df based on the session in row(exploded df)
    df_t = merged_df[merged_df[' id_session'] == row['concurrent_sessions']]
    #Filtering df_t (filtered merged_df) that only events that happend earlier than the choice time in row(exploded_df)
    filtered_df = df_t[df_t[' end_time'] < row['time']]
    filtered_df['index'] = row['index']
    #Take the max number out of the filtered_df
    closest_time = filtered_df[' end_time'].max()
    df_t_filtered = filtered_df[filtered_df[' end_time'] == closest_time]
    # Filter based on event_type
    df_t_filtered = df_t_filtered[df_t_filtered[' event_type'] == 1]
    #Assign cohice_set number based on index column in row(exploded_df)
    df_t_filtered['choice_set'] = row['index'] 
    df_t_filtered['chosen'] = row['chosen']
    df_t_filtered['wating_time'] = row['time'] -  df_t_filtered[' end_time']
    df_t_filtered['workload'] = row['workload']
    all_filtered_dfs.append(df_t_filtered)  # Append df_t_filtered to the list      
    return df_t_filtered

# Apply the max_event function to each row of df_exploded
result = df_exploded.reset_index().apply(lambda row: max_event(row), axis=1)

# Concatenate all the DataFrames in all_filtered_dfs into a single DataFrame
all_filtered_events = pd.concat(all_filtered_dfs, ignore_index=True)
# all_filtered_events.to_csv('before_analysis_2-5milion.csv')

In [ ]:
merged_df_sorted = merged_df.sort_values(by= [' id_rep', ' end_time']).reset_index()

In [ ]:
df_exploded.iloc[17000:17020]

In [ ]:
merged_df[' id_session'].dtype

In [ ]:
df_exploded['concurrent_sessions'] = df_exploded['concurrent_sessions'].astype('Int64')
merged_df['concurrent_sessions'] = merged_df['concurrent_sessions'].astype('Int64')

In [ ]:
df_exploded[df_exploded['time'] == 1494871513]

In [ ]:
df_t = merged_df[(merged_df.index >= 931124) & (merged_df.index <= 931154)]

df_t[[' id_session', ' event_type_desc', ' end_date', ' end_time']]

In [ ]:
# Filtering merged df based on the session in the current row
# session_events = merged_df.apply(lambda x:merged_df[x[' id_session'] == df_exploded['concurrent_sessions']])
# Filtering for events that happened earlier than the choice time
# past_events = session_events[session_events[' end_time'] < df_exploded['time']]

# df_filtered = merged_df[merged_df[' id_session'] == sess]



In [ ]:
df_reg['id_rep_time'] = df_reg['end_time'] + df_reg['waiting_time']
df_reg[df_reg['choice_count'] > 4][['id_session', 'event_type_desc', 'end_date', 'end_time', 'chosen', 'choice_count', 'choice_set', 'waiting_time', 'id_rep_time']].sort_values(by = 'choice_set').head(50)

In [ ]:
df_filtered = merged_df_sorted[merged_df_sorted.apply(lambda row: row[' id_session'] == 100140088, axis=1)]
time_id = 1494875525
past_events = df_filtered[df_filtered[' end_time'] < time_id]
# Find the event with the maximum end_time
closest_time = past_events[' end_time'].max()
filtered_event = past_events[past_events[' end_time'] == closest_time]
# Further filter by event_type
filtered_event = filtered_event[filtered_event[' event_type'] == 1]

filtered_event

In [ ]:
df_reg[df_reg['choice_set'] == 2410][['id_session', 'event_type_desc', 'end_date', 'end_time', 'chosen', 'choice_count', 'choice_set', 'waiting_time', 'id_rep_time']]

In [ ]:
df_t = merged_df_sorted[(merged_df_sorted.index > 19670) & (merged_df_sorted.index <= 19700)]

df_t[[' id_session', ' event_type_desc', ' end_date', ' end_time']]

In [ ]:
df_reg = pd.read_csv('df_reg_engagement_stick_with_pred_01_04.csv')

df_reg[(df_reg['id_session'] == 100022879) & (df_reg['end_time'] == 1494871025)][['id_session', 'event_type_desc', 'end_date', 'end_time', 'chosen']]

In [ ]:
merged_df_mini = merged_df[~merged_df[' id_session'].isin(df_exploded['concurrent_sessions'])]

In [ ]:
df_filtered

In [ ]:
# CHAT GPT
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# Initialize an empty list to store all filtered events
filtered_events_list = []

start_time = time.time()

def max_event(row):
    print('\r', row['index'], end='')
    # Filtering merged df based on the session in the current row
    session_events = merged_df[merged_df[' id_session'] == row['concurrent_sessions']]
    # Filtering for events that happened earlier than the choice time
    past_events = session_events[session_events[' end_time'] < row['time']]
    
    if not past_events.empty:
        # Find the event with the maximum end_time
        closest_time = past_events[' end_time'].max()
        filtered_event = past_events[past_events[' end_time'] == closest_time]
        # Further filter by event_type
        filtered_event = filtered_event[filtered_event[' event_type'] == 1]
        
        if not filtered_event.empty:
            # Add additional columns for choice_set, chosen, waiting_time, and workload
            filtered_event['choice_set'] = row['index']
            filtered_event['chosen'] = row['chosen']
            filtered_event['waiting_time'] = row['time'] - filtered_event[' end_time']
            filtered_event['workload'] = row['workload']
            
            # Append to the list
            filtered_events_list.append(filtered_event)

# Apply the function row-wise on the exploded dataframe
df_exploded.reset_index().apply(lambda row: max_event(row), axis=1)

# Concatenate all filtered events into a single DataFrame
all_filtered_events = pd.concat(filtered_events_list, ignore_index=True)

# Save the resulting DataFrame to a CSV file
all_filtered_events.to_csv('all_filtered_events_3m_f.csv', index=False)

# Display the first few rows of the resulting DataFrame
print(all_filtered_events.head())

# End timing
end_time = time.time()

# Print execution time
print(f"Execution time: {end_time - start_time:.2f} seconds")

In [ ]:
# OUR CODE


import warnings
warnings.filterwarnings('ignore')

# Initialize an empty DataFrame to store all filtered events
all_filtered_events = pd.DataFrame(columns=merged_df.columns.values)

def max_event(row):
    global all_filtered_events
    print('\r', row['index'], end='')
    # Filtering merged df based on the session in row (exploded df)
    df_t = merged_df[merged_df[' id_session'] == row['concurrent_sessions']]
    # Filtering df_t (filtered merged_df) that only events that happened earlier than the choice time in row (exploded_df)
    filtered_df = df_t[df_t[' end_time'] < row['time']]
    filtered_df['index'] = row['index']
    if not filtered_df.empty:
        # Take the max number out of the filtered_df
        closest_time = filtered_df[' end_time'].max()
        df_t_filtered = filtered_df[filtered_df[' end_time'] == closest_time]
        # Filter based on event_type
        df_t_filtered = df_t_filtered[df_t_filtered[' event_type'] == 1]
        # Assign choice_set number based on index column in row (exploded_df)
        df_t_filtered['choice_set'] = row['index']
        df_t_filtered['chosen'] = row['chosen']
        df_t_filtered['wating_time'] = row['time'] -  df_t_filtered[' end_time']
        df_t_filtered['workload'] = row['workload']
        # Append df_t_filtered to the DataFrame containing all filtered events
        all_filtered_events = all_filtered_events._append(df_t_filtered, ignore_index=True)
        
df_exploded.reset_index().apply(lambda row: max_event(row), axis=1)
all_filtered_events.to_csv('all_filtered_events_3m_a.csv')
all_filtered_events.head()

In [ ]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# Initialize an empty list to store all filtered events
filtered_events_list = []

def max_event(row):
    # Filtering merged df based on the session in the current row
    session_events = merged_df[merged_df[' id_session'] == row['concurrent_sessions']]
    # Filtering for events that happened earlier than the choice time
    past_events = session_events[session_events[' end_time'] < row['time']]
    
    if not past_events.empty:
        # Find the event with the maximum end_time
        closest_time = past_events[' end_time'].max()
        filtered_event = past_events[past_events[' end_time'] == closest_time]
        # Further filter by event_type
        filtered_event = filtered_event[filtered_event[' event_type'] == 1]
        
        if not filtered_event.empty:
            # Add additional columns for choice_set, chosen, waiting_time, and workload
            filtered_event['choice_set'] = row['index']
            filtered_event['chosen'] = row['chosen']
            filtered_event['waiting_time'] = row['time'] - filtered_event[' end_time']
            filtered_event['workload'] = row['workload']
            
            # Append to the list
            filtered_events_list.append(filtered_event)

# Apply the function row-wise on the exploded dataframe
df_exploded.reset_index().apply(lambda row: max_event(row), axis=1)

# Concatenate all filtered events into a single DataFrame
all_filtered_events = pd.concat(filtered_events_list, ignore_index=True)

# Save the resulting DataFrame to a CSV file
all_filtered_events.to_csv('all_filtered_events_3m_f.csv', index=False)

# Display the first few rows of the resulting DataFrame
print(all_filtered_events.head())

In [ ]:
import pandas as pd
import time
import warnings

warnings.filterwarnings('ignore')

# Start timing
start_time = time.time()

# STEP 1: Preprocess merged_df
# Make sure to strip column names (remove extra spaces)
merged_df.columns = merged_df.columns.str.strip()
df_exploded.columns = df_exploded.columns.str.strip()

# Sort for efficient filtering
merged_df = merged_df.sort_values(['id_session', 'end_time'])

# Group by session for fast access
session_groups = dict(tuple(merged_df.groupby('id_session')))

# List to collect results
filtered_events_list = []

# STEP 2: Iterate over df_exploded
for row in df_exploded.reset_index().itertuples(index=False):
    session_events = session_groups.get(row.concurrent_sessions, pd.DataFrame())

    if session_events.empty:
        continue

    # Get only events before the choice time
    past_events = session_events[session_events['end_time'] < row.time]

    if past_events.empty:
        continue

    # Get the latest event
    closest_time = past_events['end_time'].max()
    filtered_event = past_events[past_events['end_time'] == closest_time]

    # Keep only event_type == 1
    filtered_event = filtered_event[filtered_event['event_type'] == 1]

    if not filtered_event.empty:
        # Enrich the row
        filtered_event = filtered_event.assign(
            choice_set=row.index,
            chosen=row.chosen,
            waiting_time=row.time - filtered_event['end_time'],
            workload=row.workload
        )

        filtered_events_list.append(filtered_event)

import pickle

# Save
with open("filtered_events_list.pkl", "wb") as f:
    pickle.dump(filtered_events_list, f)

# Later (after restarting kernel) reload
with open("filtered_events_list.pkl", "rb") as f:
    filtered_events_list = pickle.load(f)




In [ ]:
import pickle
with open("filtered_events_list.pkl", "rb") as f:
    filtered_events_list = pickle.load(f)

In [ ]:
import os, sys

path = "filtered_events_list.pkl"
print("CWD:", os.getcwd())
print("Exists:", os.path.exists(path))
print("Size:", os.path.getsize(path) if os.path.exists(path) else "N/A")


In [ ]:
import pickle, os

def save_whole_list(filtered_events_list, path="filtered_events_list.pkl"):
    tmp = path + ".tmp"
    with open(tmp, "wb") as f:
        pickle.dump(filtered_events_list, f, protocol=pickle.HIGHEST_PROTOCOL)
        f.flush()
        os.fsync(f.fileno())
    os.replace(tmp, path)  # atomic rename

# usage
save_whole_list(filtered_events_list)


In [ ]:
filtered_events_list

In [ ]:
# Now you can concat in a fresh session
all_filtered_events = pd.concat(filtered_events_list, ignore_index=True)

In [ ]:
import pandas as pd
import time
import warnings
import os

warnings.filterwarnings('ignore')

# Start timing
start_time = time.time()

# Preprocess merged_df and df_exploded
merged_df.columns = merged_df.columns.str.strip()
df_exploded.columns = df_exploded.columns.str.strip()

merged_df = merged_df.sort_values(['id_session', 'end_time'])
session_groups = dict(tuple(merged_df.groupby('id_session')))

# Output file
output_file = "all_filtered_events_3m_f.csv"
first_write = True  # to write header only once

# Loop over exploded df
for row in df_exploded.reset_index().itertuples(index=False):
    session_events = session_groups.get(row.concurrent_sessions, pd.DataFrame())
    if session_events.empty:
        continue

    past_events = session_events[session_events['end_time'] < row.time]
    if past_events.empty:
        continue

    closest_time = past_events['end_time'].max()
    filtered_event = past_events[past_events['end_time'] == closest_time]
    filtered_event = filtered_event[filtered_event['event_type'] == 1]

    if not filtered_event.empty:
        filtered_event = filtered_event.assign(
            choice_set=row.index,
            chosen=row.chosen,
            waiting_time=row.time - filtered_event['end_time'],
            workload=row.workload
        )

        # ✅ Instead of append → write directly to CSV
        filtered_event.to_csv(
            output_file,
            mode='a',
            header=first_write,
            index=False
        )
        first_write = False  # only write header the first time

# Done
end_time = time.time()
print(f"Execution time: {end_time - start_time:.2f} seconds")
print(f"Results saved to {output_file}")



-------------------

In [ ]:
df_new = pd.read_csv('all_filtered_events_all_data_26_08.csv')

df_new

In [ ]:
df_old = pd.read_csv('df_all_24_06_with_after_last_and_not_chosen.csv')

df_old

In [ ]:
#df_new.drop(131842, inplace= True)

df_new.reset_index(drop = True)

In [ ]:
df_new['first_msg_dummy'] = df_new['first_msg_dummy'].astype(int)

df_new['first_msg_dummy'].value_counts()

Approximately 1 million events are first messages that were sent in a session (i.e, messages that initiated the conversation before an assignment to any agent)

In [ ]:
df_new[(df_new['first_msg_dummy'] == 1) & (df_new['event_id'].duplicated())]

Checking validity of new choice model dataset

In [ ]:
df_raw = pd.read_csv('merged_df_after_dan_6m_renewal_16_08_FULL.csv')

df_raw

In [ ]:
df_new.sort_values(by = ['id_rep', 'end_time'], inplace= True)

df_new

In [ ]:
df_new[df_new['choice_set'] == 6311]

In [ ]:
df_raw = df_raw.sort_values(by = [' id_rep', ' end_time'])
df_raw[(df_raw[' id_rep'] == 30000005.0) & (df_raw[' end_time'] > 1493693910) & (df_raw[' end_time'] < 1493694290)]

In [ ]:
df_raw[(df_raw['first_msg_dummy'] == 1) & (df_raw['assignment_date'].isna())].shape

In [ ]:
df_new['id_session'].nunique()

-----------------------------

In [ ]:
all_filtered_events = pd.read_csv('all_filtered_events_all_data_26_08.csv')


In [ ]:
all_filtered_events.head(50)

In [ ]:
all_filtered_events[' id_session'].nunique()

In [ ]:
all_filtered_events[' end_date'].max()

In [ ]:
all_filtered_events["choice_set"].nunique()

In [ ]:
all_filtered_events

In [ ]:
all_filtered_events['id_session'].nunique()- df['id_session'].nunique()

In [ ]:
all_filtered_events['choice_set'].nunique() - df['choice_set'].nunique()

In [ ]:
# all_filtered_events['index'].iloc[0:50]
df = all_filtered_events.loc[all_filtered_events.duplicated(subset='choice_set', keep=False)]
df = df.reset_index(drop=True)

df.shape

In [ ]:
df['id_session'].nunique()

In [ ]:
#A function that turns a column into an int:
df['sentiment'] = df['sentiment'].astype(int)
df['number_words'] = df['number_words'].astype(int)
df['waiting_time'] = df['waiting_time'].astype(int)
df['chosen'] = df['chosen'].astype(int)
df['workload'] = df['workload'].astype(int)
df.head()

In [ ]:
pd.options.mode.chained_assignment = None
# Creation of control variables: FCFS, Stickiness, hour
#Stickness:
df_t = pd.DataFrame()
for index in range(len(df)):
    if 'stickiness' not in df.columns:
        df['stickiness'] = 0
    var = df['chosen'][index] 
    if var  == 1.0:
        session = df.iloc[[index]]
        df_t = df_t._append(session)
        df_t = df_t.reset_index(drop = True)
        if len(df_t) == 2:
            first_row = df_t.at[0, 'id_session']
            # print(first_row)
            second_row = df_t.at[1, 'id_session']
            # print(second_row)
            if first_row == second_row:
                df['stickiness'][index] = 1
            else:
                df['stickiness'][index] = 0
        elif len(df_t) > 2:
                df_t = pd.DataFrame()
    else:
        df['stickiness'][index] = 0
        continue

In [ ]:
final_df_cl['stickiness'].value_counts()

In [ ]:
final_df_cl.head()

In [ ]:
final_df_cl.to_csv('final_df_after_fcfs.csv')

In [ ]:
#FCFS:
min_values = df.groupby('choice_set')[' end_time'].transform('min')

# Assign 1 to the rows where the value equals the minimum value in the group, and 0 otherwise:
df['FCFS'] = (df[' end_time'] == min_values).astype(int)
# Checking if FCFS works properly
df[['id_visitor', 'chosen', 'stickiness', 'choice_set']][:30]

In [ ]:
# How many choice sets are we left with:
df.shape

In [ ]:
final_df_cl = pd.read_csv('Final_regression_all_data_09_03.csv')

final_df_cl

In [ ]:
# Hour:
# Convert Unix timestamps to datetime objects
final_df_cl['datetime'] = pd.to_datetime(final_df_cl['end_time'], unit='s')

# Round the datetime objects to the nearest hour
final_df_cl['rounded_datetime'] = final_df_cl['datetime'].dt.round('H').dt.hour
final_df_cl.head()

In [ ]:
final_df_cl['rounded_datetime'] = final_df_cl['rounded_datetime'].astype('category')
final_df_cl['rounded_datetime'].tail()

In [ ]:
final_df_cl['rounded_datetime'].hist(bins = 100)
plt.title('Time of day')
plt.xlabel('Rounded hour') 
plt.ylabel('Number of events')

In [ ]:
final_df_cl.to_csv('regression_raw_before_clean_03_09.csv', index = 0)

In [ ]:
df_2m_end = final_df_cl
df_2m = pd.read_csv('regression_2m_FINAL.csv')

In [ ]:
a = df_2m[['choice_set', 'index']]

(a['choice_set'] != a['index']).sum()

In [ ]:
df_2m['choice_set'].nunique()

In [ ]:
df_2m_end['choice_set'].nunique()

In [ ]:
# df_2m['choice_set_new'] = df_2m['choice_set'] + 260609
# df_2m['choice_set'] = df_2m['choice_set_new']
# df_2m.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0', 'index', 'wating_time'], inplace= True)
df_2m.columns.shape

In [ ]:
df_2m_end['choice_set'].head()

# Checking both files have the same number of columns

In [ ]:
# Adding choice set number so it would be ready for merge with the file describing previeous events
df_2m_end['choice_set'] = df_2m_end['choice_set'] + 533988

df_2m_end['choice_set'].head()

In [ ]:
# MERGING the 2 dataframes

df = df_2m._append(df_2m_end)

df['workload'].hist(bins = 20)
# plt.xticks(np. arange(2, 12, step=1))
plt.title('Concurrent sessions')
plt.xlabel('Number of sessions') 
plt.ylabel('Count')

In [ ]:
df['choice_set'].nunique()

In [ ]:
df.to_csv('Final_regression_all_data.csv')

In [ ]:
df = pd.read_csv('Final_regression_all_data.csv')
df.shape

In [ ]:
df.columns

In [ ]:
df[[' id_session', ' id_rep',  'choice_set',
    ' start_date',  ' end_date', ' end_time', ' sentiment', ' number_words', 
    "workload", "FCFS", "stickiness", 'chosen']][:100]

In [ ]:
df = final_df_cl

CLEANING DATA

In [ ]:
# Check if every choice_set has exactly one '1' in the 'chosen' column
choice_set_validation = df.groupby('choice_set')['chosen'].sum()

# Identify problematic choice sets
invalid_choice_sets = choice_set_validation[choice_set_validation < 1].index

# Print invalid choice sets for inspection
if not invalid_choice_sets.empty:
    print("Invalid choice sets found:")
    print(df[df['choice_set'].isin(invalid_choice_sets)])
else:
    print("All choice sets are valid!")

In [ ]:
invalid_df = df[df['choice_set'].isin(invalid_choice_sets)]

invalid_df

In [ ]:
valid_df = df[~df['choice_set'].isin(invalid_choice_sets)]
#valid_df.drop(columns = "Unnamed: 0", inplace = True)
valid_df.reset_index(drop = True, inplace= True)

valid_df.shape

REMOVAL OF OUTLIERS - WAITING TIME

In [ ]:
# Get the choice_set values of the outliers
outlier_choice_sets = valid_df.loc[valid_df['waiting_time'] >= 2995, 'choice_set'].unique()
# Delete all rows belonging to these choice_sets from the original dataframe
filtered_df = valid_df[valid_df["choice_set"].isin(outlier_choice_sets)]
print(f"All THE EVENTS OF CHOICE SETS WITH AT LEAST ONE OUTLIER, N = {filtered_df.shape[0]}, number of choice sets -> {filtered_df['choice_set'].nunique()}")
# Delete all choisets that are with 2 events:
# Group by choice set
df_t = filtered_df.groupby("choice_set").count()
# Delete:
# All choice sets that we want to delete because they are ruined due to filtering, since n = 2
df_a = df_t.loc[df_t['waiting_time'] == 2]
# Deleting said choice sets
valid_df_2 = valid_df[~valid_df["choice_set"].isin(df_a.index)]
print(f"The number of erased events becuase n of choice set equals 2: {valid_df.shape[0] - valid_df_2.shape[0]}")
print(f"The number of erased choice sets becuase n of choice set equals 2: {valid_df['choice_set'].nunique() - valid_df_2['choice_set'].nunique()}")
df_b = df_t.loc[df_t['waiting_time'] > 2]

filtered_df = valid_df[valid_df["choice_set"].isin(df_b.index)]
outlier_choice_sets_2 = filtered_df.loc[(filtered_df['waiting_time'] > 2995) & (filtered_df['chosen'] == 1), 'choice_set'].unique()
valid_df_3 = valid_df_2[~valid_df_2["choice_set"].isin(outlier_choice_sets_2)]
print(f'The number of erased events becuase outlier event is chosen: {valid_df_2.shape[0] - valid_df_3.shape[0]}')
print(f"The number of erased choice sets becuase outlier event is chosen: {valid_df_2['choice_set'].nunique() - valid_df_3['choice_set'].nunique()}")

In [ ]:
valid_df_3 = valid_df_3.loc[valid_df_3["waiting_time"] < 2995].reset_index(drop = True)

In [ ]:
list_1 = valid_df_3["waiting_time"].groupby(valid_df_3["choice_set"]).count()
list_1 = pd.DataFrame(list_1)
list_1["choice_set"] = list_1.index
list_1 = list_1.reset_index(drop = True)
list_1.rename(columns = {"waiting_time" : "num_events"}, inplace = True)
df_one_events = list_1[list_1["num_events"] == 1]
df_one_events
df_t = valid_df_3[~valid_df_3["choice_set"].isin(df_one_events["choice_set"])]
df_t.reset_index(drop = True, inplace = True)

In [ ]:
df_t.to_csv('only_waiting_time_cleaned_03_09.csv', index = 0)

In [ ]:
filtered_df = pd.read_csv('only_waiting_time_cleaned_03_09.csv', index_col=0)
filtered_df.head()

REMOVAL OF OUTLIERS - NUMBER OF WORDS

In [ ]:
# Get the choice_set values of the outliers
outlier_choice_sets_num = filtered_df.loc[filtered_df['number_words'] >= 900, 'choice_set'].unique()

# Delete all rows belonging to these choice_sets from the original dataframe
df_reg = filtered_df[~filtered_df["choice_set"].isin(outlier_choice_sets_num)]
print(f'This is the number of erased rows: {len(filtered_df) - len(df_reg)}')

In [ ]:
# 4.12 - 814689 rows are documented
df_reg.reset_index(drop = True, inplace = True)
df_reg.to_csv('before_regression_full_data_galit_03_09.csv', index = 0)

In [ ]:
df_reg = pd.read_csv('before_regression_full_data_galit_03_09.csv')
df_reg.shape

REGRESSION

In [ ]:
#Regression anlysis:
df_reg.reset_index(inplace = True, drop = True)
xtrain = df_reg[['waiting_time', ' number_words', ' sentiment', 'workload', 'waiting_time_squared']]
ytrain = df_reg[['chosen']]

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor 
# VIF dataframe 
vif_data = pd.DataFrame() 
vif_data["feature"] = xtrain.columns 
  
# calculating VIF for each feature 
vif_data["VIF"] = [variance_inflation_factor(xtrain.values, i) 
                          for i in range(len(xtrain.columns))] 
  
print(vif_data)

In [ ]:
xtrain.corr()

In [ ]:
# Scaling for convergence of model
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
xtrain_scaled = scaler.fit_transform(xtrain)

xtrain_scaled

In [ ]:
# Transforming numpy array to dataframe with column names
xtrain_df = pd.DataFrame(xtrain_scaled)
xtrain_df.columns = xtrain.columns

xtrain_df

In [ ]:
xtrain_df.std().values

In [ ]:
from statsmodels.discrete.conditional_models import ConditionalLogit
from sklearn.preprocessing import StandardScaler
g = df_reg['choice_set']

# Fit the Conditional Logit model
try:
    cl_model = ConditionalLogit(endog=ytrain, exog=xtrain_df, groups=g)
    cl_fit = cl_model.fit(maxiter=1000)  # Increase iterations
    print(cl_fit.summary())
except Exception as e:
    print(f"Error fitting model: {e}")

Addition of Non - linear effect

In [ ]:
df_reg['waiting_time_squared'] = (df_reg['waiting_time'])**2

df_reg['waiting_time_squared']

In [ ]:
std_array = xtrain.std().values
coef_array = cl_fit.params.values
names_array = cl_fit.params.index

def effect_size_logit(name, coef, std):
    df_t = pd.DataFrame({'name': name ,'std': std, 'coef': coef})
    result = np.exp(df_t['coef'])
    result_exp = []
    for coef_row in result:
        if coef_row < 1:
            result_val = 1 - coef_row
        elif coef_row > 1:
            result_val = coef_row - 1
        else:
            result_val = coef_row
        result_exp.append(result_val)
    df_t['before_sted'] = result_exp
    df_t['std'].astype('float64')
    df_t['effect_size'] = df_t['before_sted'] / df_t['std']
    return df_t

df_effect_size = effect_size_logit(names_array, coef_array, std_array)
df_effect_size

In [ ]:
df_reg.columns

NOTES

In [ ]:
'''

- Waiting Time:

Significant influence on being chosen, with an effect size of: -30%, as in, every 5 the chance to get chosen decreases in around 30%

- Number_words:

Significant influence on being chosen, with an effect size of: -4.4%, as in, every additional character decreases the chance to get chosen.
This confirms our hypothesis regarding complexity and being chosen.

- FCFS:

Significant influence on being chosen, with an effect size of: 14%, as in, if the customer was the first one to enter the choice set,
it increases his chance to get chosen in 14%.

- Stickiness:

Non significant effect.











'''

In [ ]:
df_reg = pd.read_csv("before_regression_full_data_threshold_outliers_07_01.csv")
df_reg.head()

20.12 working on new variable of last message in conversation

In [ ]:
merged_df = pd.read_csv('merged_df_after_dan_6m_renewal_16_08_FULL.csv', index_col = 0)
merged_df.shape

In [ ]:
merged_df= merged_df[[" id_visitor", ' id_session', " id_rep", " start_date", " start_time"," end_date",
                       " end_time"," duration", " event_type", " event_type_desc", ' sentiment', ' number_words']]
#sort by time:
merged_df = merged_df.sort_values(by=[' id_rep', ' end_time' ])
merged_df = merged_df.reset_index(drop=True)
merged_df.shape

In [ ]:
#All the messages that were sent by only customer:
y = merged_df.loc[merged_df[" event_type"] == 1]
y_max_time = y[y.groupby(" id_session")[' end_date'].transform("max") == y[' end_date']]
y_max_time.reset_index(drop = True, inplace = True)

In [ ]:
y = y_max_time

In [ ]:
y['id_v'] = y[" id_visitor"].astype("str")
y['end_t'] = y[" end_time"].astype("str")
y["number_w"] = y[" number_words"].astype("str")

In [ ]:
# df_reg = pd.read_csv("before_regression_full_data_galit__12.csv")
df_reg.shape

In [ ]:
y["for_merge"] = y['end_t'].str.cat(y['id_v'])
# y["for_merge"] = y["for_merge"].str.cat(y["number_w"])
y["for_merge"].nunique()

In [ ]:
df_reg['id_v'] = df_reg["id_visitor"].astype("str")
df_reg['end_t'] = df_reg["end_time"].astype("str")
# df_reg["number_w"] = df_reg[" number_words"].astype("str")

df_reg["for_merge"] = df_reg['end_t'].str.cat(df_reg['id_v'])
# df_reg["for_merge"] = df_reg["for_merge"].str.cat(df_reg["number_w"])
df_reg["for_merge"].nunique()

In [ ]:
df_last = df_reg[df_reg['for_merge'].isin(y['for_merge'])]
df_last.shape

In [ ]:
# Creation of new column that shows last messages of conversations (sessions)
df_reg['last_msg'] = df_reg['for_merge'].isin(df_last['for_merge']).astype(int)

# Checking the number of last messages is identical to rows we found previously
df_reg.loc[df_reg['last_msg'] == 1].shape

In [ ]:
df_reg.head()

In [ ]:
df_reg.to_csv('before_regression_full_data_31_12.csv')

In [ ]:
df_reg = pd.read_csv('before_regression_full_data_24_12.csv', index_col = 0)

df_reg

22.12 - Creation of number of messages untill choosing event variable

In [ ]:
df_customers = merged_df.loc[merged_df[" event_type"] == 1]

df_customers.shape

In [ ]:
# df_customers['count_msg'] = 0
df_customers.reset_index(drop = True, inplace =True)

In [ ]:
df_customers = df_customers.sort_values(by = " end_time")
df_customers['msg_count'] = df_customers.groupby(by=[' id_session'])[" end_time"].transform(lambda x: x.rank())

In [ ]:
df_customers[[" id_session"," end_time", "msg_count"]].sort_values([" id_session", " end_time"])

In [ ]:
df_customers['id_v'] = df_customers[" id_visitor"].astype("str")
df_customers['end_t'] = df_customers[" end_time"].astype("str")
#df_customers["number_w"] = df_customers[" number_words"].astype("str")
df_customers["for_merge"] = df_customers['end_t'].str.cat(df_customers['id_v'])

In [ ]:
df_reg['for_merge'] = df_reg['for_merge'].astype("str")

In [ ]:
tmp = df_customers[['for_merge', 'msg_count']]
df_reg = df_reg.merge(tmp, on= 'for_merge')

In [ ]:
print(df_customers[df_customers['for_merge'] == '1496270947200021764']['msg_count'])
print(df_reg[df_reg['for_merge'] == '1496270947200021764']['msg_count'])

In [ ]:
df_reg.loc[df_reg[" id_session"] == 100045597].sort_values(by = " end_time")

In [ ]:
df_reg.columns

In [ ]:
df_reg.head(100)[["id_session", "end_date", "msg_count"]].sort_values(["end_date", "id_session"])

25.12 Creation of percentage of messages untill event

In [ ]:
# Creating a column of total messages of conversation for each customer
df_temp = df_customers[['msg_count', ' id_session']].groupby(by = ' id_session').max() 
df_temp[' id_session'] = df_temp.index
df_temp.rename(columns = {"msg_count" : "msg_max"}, inplace = True)
df_temp.reset_index(drop = True, inplace = True)
df_temp
x = df_reg.merge(df_temp, left_on= "id_session", right_on= " id_session")
x

In [ ]:
x['msg_perc'] = x['msg_count'] / x['msg_max']

x['msg_perc'].describe()

In [ ]:
df_reg = x
df_reg.head()
df_reg.to_csv('before_regression_full_data_msg_perc_count_max_04_09.csv', index = 0)

In [ ]:
df_reg[['msg_perc', 'chosen']].groupby(by = "chosen").mean()

In [ ]:
df_customers[df_customers[' id_session'] == 100102591].sort_values(by = ' end_time')

Application of same methods to number words - sum and perc of number_words per msg for each session

Sum of number_words per msg for each session

In [ ]:
merged_df = pd.read_csv('merged_df_after_dan_6m.csv', index_col = 0)
merged_df.shape

# df_reg = pd.read_csv('before_regression_full_data_31_12.csv', index_col = 0)


In [ ]:
df_customers = merged_df.loc[merged_df[" event_type"] == 1]
df_customers.reset_index(drop = True, inplace =True)

df_customers = df_customers.sort_values(by = " end_time")
df_customers['number_words_cum_sum'] = df_customers.groupby(by=[' id_session'])[" number_words"].transform(lambda x: x.cumsum())
# df_customers[[" id_session"," end_time", ' number_words', "number_words_cum_sum"]].sort_values([" id_session", " end_time"])

In [ ]:
df_customers.columns

In [ ]:
df_customers['id_v'] = df_customers[" id_visitor"].astype("str")
df_customers['end_t'] = df_customers[" end_time"].astype("str")
df_customers["number_w"] = df_customers[" number_words"].astype("str")
df_customers["for_merge"] = df_customers['end_t'].str.cat(df_customers['id_v'])

In [ ]:
#Creation of overall sum:
df_customers['number_words_sum'] = df_customers.groupby(by=[' id_session'])[" number_words"].transform(lambda x: x.sum())
df_customers[[" id_session"," end_time", ' number_words', "number_words_sum"]].sort_values([" id_session", " end_time"])[:100]

In [ ]:
#Creation of perc column:
df_customers["number_words_perc"] = df_customers["number_words_cum_sum"] / df_customers["number_words_sum"]
df_customers[[" id_session"," end_time", 'number_words_cum_sum', "number_words_sum", "number_words_perc"]].sort_values([" id_session", " end_time"])[:100]

In [ ]:
df_reg["for_merge"] = df_reg["for_merge"].astype("str")

In [ ]:
df_customers.columns

In [ ]:
# Merging the variables into df_reg:
tmp = df_customers[['for_merge', 'number_words_sum', "number_words_cum_sum", "number_words_perc"]]
df_reg_temp = df_reg.merge(tmp, on= 'for_merge')

In [ ]:
df_reg_temp[[' number_words', 'number_words_sum', "number_words_cum_sum", "number_words_perc", ' id_session', ' end_time']].sort_values([' id_session', ' end_time'])

In [ ]:
df_reg_temp[['number_words_sum', 'number_words_perc', 'number_words_cum_sum']].groupby(df_reg_temp['chosen']).mean()

In [ ]:
df_reg_temp.shape

In [ ]:
df_reg_temp.to_csv('before_regression_full_data_thresold_outliers_all_variables.csv')

Creation of binary sentiment variable

In [ ]:
df_reg[' sentiment'].where>0

In [ ]:
df_reg['binary_sentiment']= np.where((df_reg[' sentiment'] > 0, 1, 0)

In [ ]:
def categorial_sentiment_func (x):
    if x > 0.0:
        x = 1
    elif x == 0.0:
        x = 0
    else:
        x = -1
    return x

In [ ]:
df_reg_temp['categorial_sentiment'] = df_reg_temp[' sentiment'].apply(lambda x: categorial_sentiment_func (x))

df_reg_temp[df_reg_temp['categorial_sentiment'] < 0][[' sentiment', 'categorial_sentiment']]

In [ ]:
df_reg_temp.to_csv('before_regression_full_data_thresold_outliers_all_variables.csv')

In [ ]:
df_reg = pd.read_csv('before_regression_full_data_thresold_outliers_all_variables.csv')

In [ ]:
df_reg.groupby("FCFS")[["chosen"]].count()

In [ ]:
df_reg_temp[" id_visitor"].nunique() / df_reg_temp[" id_rep"].nunique() 

In [ ]:
df_reg_temp[" id_session"].nunique() / merged_df[" id_session"].nunique()

In [ ]:
merged_df[[" id_session", " id_rep", " id_visitor"," event_type", " end_date"]].head(500).sort_values(" end_date")

In [ ]:
df_reg_temp.columns

In [ ]:
df = pd.read_csv('before_regression_full_data_31_12.csv')

In [ ]:
df["waiting_time"].describe()

In [ ]:
x[[" id_visitor"," end_time","msg_count"]].sort_values([" id_visitor", " end_time"])

In [ ]:
df_reg_temp.columns

In [ ]:
df_reg_temp['number_words_cum_sum'].describe()

In [ ]:
df_reg_temp['msg_count'].describe()

In [ ]:
df_reg_temp['msg_perc'].describe()

In [ ]:
df_reg_temp['msg_max'].describe()

In [ ]:
df_reg_temp['number_words_perc'].describe()

In [ ]:
df_reg_temp['number_words_sum'].describe()

30.12 zooming into the events that caused difference in waiting_time coef

In [ ]:
df_less = pd.read_csv('before_regression_full_data_11_12.csv')
df_more = pd.read_csv('before_regression_full_data_galit_20_12.csv')

In [ ]:
df_less['id_v'] = df_less[" id_visitor"].astype("str")
df_less['end_t'] = df_less[" end_time"].astype("str")
df_less['choice_s'] = df_less["choice_set"].astype("str")
# df_less["number_w"] = df_less[" number_words"].astype("str")

df_less["for_merge"] = df_less['end_t'].str.cat(df_less['id_v'])
df_less["for_merge"] = df_less['for_merge'].str.cat(df_less['choice_s'])
# df_less["for_merge"] = df_less["for_merge"].str.cat(df_less["number_w"])
print(df_less["for_merge"].nunique())

df_more['id_v'] = df_more[" id_visitor"].astype("str")
df_more['end_t'] = df_more[" end_time"].astype("str")
df_more['choice_s'] = df_more["choice_set"].astype("str")
# df_less["number_w"] = df_less[" number_words"].astype("str")

df_more["for_merge"] = df_more['end_t'].str.cat(df_more['id_v'])
df_more["for_merge"] = df_more['for_merge'].str.cat(df_more['choice_s'])
# df_less["for_merge"] = df_less["for_merge"].str.cat(df_less["number_w"])
df_more["for_merge"].nunique()

In [ ]:
df_more['for_merge'] = df_more['for_merge'].astype("str")

print(df_more['for_merge'].dtypes)

df_less['for_merge'] = df_less['for_merge'].astype("str")

x = df_less['for_merge'].dtypes
print(f'df_less for_merge is {x}')

In [ ]:
df_more

In [ ]:
df_last = df_more[~df_more['for_merge'].isin(df_less['for_merge'])]
df_last.shape

In [ ]:
df_outliers = df_last.reset_index(drop = True)
df_outliers.to_csv("df_outliers_31_12.csv", index = 0)

In [ ]:
df_more['waiting_time'].describe()

In [ ]:
df_more["waiting_time"].hist(bins = 50)

In [ ]:
df_less["waiting_time"].hist(bins = 50)

----------------------------------------------

In [ ]:
# Reading csv's
df = pd.read_csv("before_regression_full_data_thresold_outliers_all_variables.csv", index_col=0)
agents_session =  pd.read_csv("merged_agents_session.csv")
session_append =  pd.read_csv("sessions_append.csv", index_col=0)

In [ ]:
merged_session[merged_session[' agent_status'] == 3]

In [ ]:
merged_session[' end_date'] = pd.to_datetime(merged_session[' end_date'], format = 'mixed')
merged_session['day_count'] = pd.factorize(merged_session[' end_date'].dt.date)[0] + 1

merged_session

In [ ]:
filtered_login_merged_session = merged_session[merged_session[' agent_status'] == 1]
filtered_logout_merged_session = merged_session[merged_session[' agent_status'] == 5]
# merged_session['start_shift'] = filtered_login_merged_session.groupby([' id_rep', 'day_count']).agg({' end_date': 'min'})
# merged_session['end_shift'] = filtered_logout_merged_session.groupby([' id_rep', 'day_count']).agg({' end_date': 'max'}.reset_index())

In [ ]:
start_shift = (
    filtered_login_merged_session.groupby([' id_rep', 'day_count'])[' end_date']
    .min()
    .reset_index()
    .rename(columns={' end_date': 'start_shift'})
)

# Compute end_shift
end_shift = (
    filtered_logout_merged_session.groupby([' id_rep', 'day_count'])[' end_date']
    .max()
    .reset_index()
    .rename(columns={' end_date': 'end_shift'})
)

# Merge the computed shifts back to the original DataFrame
merged_session = merged_session.merge(start_shift, on=[' id_rep', 'day_count'], how='left')
merged_session = merged_session.merge(end_shift, on=[' id_rep', 'day_count'], how='left')

merged_session

In [ ]:
merged_session = pd.read_csv('agents_shift_15_2.csv')

merged_session

In [ ]:
merged_session['logout'] = pd.to_datetime(merged_session['logout'], format = 'mixed')
# df_t = merged_session[merged_session[' agent_status'] == 5]
merged_session['logout'] = merged_session['logout'].dt.round('H').dt.hour

In [ ]:
merged_session['logout'].hist(bins = 50)

In [ ]:
merged_session['log_in'] = pd.to_datetime(merged_session['log_in'], format = 'mixed')
# df_t = merged_session[merged_session[' agent_status'] == 5]
merged_session['log_in'] = merged_session['log_in'].dt.round('H').dt.hour

merged_session['log_in'].hist(bins = 50)

In [ ]:
merged_session.groupby(id_rep)

In [ ]:
def cat1(value):
    if value == 0:
        return "Not chosen"
    elif value == 1:
        return "Chosen"
    else:
        None
df['chosen_string'] = df.apply(lambda row: cat1(row['chosen']), axis = 1)
df.groupby('chosen_string')['waiting_time'].mean().plot(kind = 'bar', title = 'Difference in waiting time between being chosen or not', rot = 0)

In [ ]:
df.groupby('chosen')[' sentiment'].describe()

In [ ]:
df.groupby('chosen_string')['categorial_sentiment'].count().plot(kind = 'bar', title = 'Difference in sentiment between being chosen or not', rot = 0)

In [ ]:
pd.crosstab(df['chosen_string'], df['categorial_sentiment']).plot(kind = 'bar', rot = 0, title = 'Difference in sentiment between being chosen or not')

In [ ]:
pd.crosstab(df['chosen_string'], df['categorial_sentiment'])

In [ ]:
df_t['terminated_by'].value_counts()

### Session termination type

In [ ]:
df.rename(columns = {' id_session': 'id_session'}, inplace = True)

df.columns

In [ ]:
df_t = session_append[['id_session', 'terminated_by']]
df_right = df_t.drop_duplicates(subset='id_session')

In [ ]:
df_temp = df.merge(df_right, on = 'id_session', how = 'left')

df_temp['terminated_by'].value_counts()

In [ ]:
df_temp.shape

#### Creating terminated by types, attaching numbers to categorial values

In [ ]:
def cat2(value):
    if value == 1:
        return "System"
    elif value == 2:
        return "Agent"
    elif value == 3:
        return "Customer"
    elif value == 4:
        return "Manager"
    else:
        None
df_temp['terminated_by_string'] = df_temp.apply(lambda row: cat2(row['terminated_by']), axis = 1)

In [ ]:
df_temp['terminated_by_string'].value_counts().plot(kind = 'bar', rot = 0, title = 'Session termination type')

In [ ]:
df_temp['terminated_by_string'].value_counts().plot(kind = 'bar', rot = 0, title = 'Session termination type')

In [ ]:
pd.crosstab(df_temp['chosen_string'], df_temp['terminated_by_string'])

In [ ]:
df_temp.to_csv('df_reg_before_quantiles_20_2.csv')

# Split to levels - Waiting time, msg_perc, number_words

In [ ]:
# df_temp = pd.read_csv('df_reg_before_quantiles_20_2.csv')

df_temp = df_reg

df_temp

## Waiting Time

In [ ]:
wt_quantile1 = df_temp['waiting_time'].quantile(.20)
wt_quantile2 = df_temp['waiting_time'].quantile(.40)
wt_quantile3 = df_temp['waiting_time'].quantile(.60)
wt_quantile4 = df_temp['waiting_time'].quantile(.80)
def variable_levels (x, df, variable, quantile1, quantile2, quantile3, quantile4):
    if x < quantile1:
        return 'fifth_1'
    elif (x >= quantile1) & (x < quantile2):
        return 'fifth_2'
    elif (x >= quantile2) & (x < quantile3):
        return 'fifth_3'
    elif (x >= quantile3) & (x < quantile4):
        return 'fifth_4'
    elif x >= quantile4:
        return 'fifth_5'
    else:
        None

df_temp['waiting_time_fifths'] = df_temp['waiting_time'].apply(lambda x: variable_levels(x, df_temp, 'waiting_time',
                                                                                          wt_quantile1, wt_quantile2, wt_quantile3, wt_quantile4))

df_temp[['id_session', 'waiting_time', 'waiting_time_fifths']]

In [ ]:
df_temp['waiting_time_fifths'].value_counts()

In [ ]:
df_temp['waiting_time_fifths'].value_counts()

In [ ]:
df_temp = df_reg

## Message percent

In [ ]:
msg_perc_quantile1 = df_temp['msg_perc'].quantile(.20)
msg_perc_quantile2 = df_temp['msg_perc'].quantile(.40)
msg_perc_quantile3 = df_temp['msg_perc'].quantile(.60)
msg_perc_quantile4 = df_temp['msg_perc'].quantile(.80)
df_temp['msg_perc_fifths'] = df_temp['msg_perc'].apply(lambda x: variable_levels(x, df_temp, 'msg_perc', msg_perc_quantile1, msg_perc_quantile2,
                                                                                  msg_perc_quantile3, msg_perc_quantile4))

df_temp[['id_session', 'msg_perc', 'msg_perc_fifths', 'msg_count', 'msg_max']]

In [ ]:
df_temp['msg_perc_fifths'].value_counts()

In [ ]:
df_temp['msg_perc'].hist(bins = 20)
plt.xticks(np.arange(0, 1.1, 0.1))
plt.title('Frequency of message percent out of entire conversation')

## Number of words

In [ ]:
number_words_quantile1 = df_temp['number_words'].quantile(.20)
number_words_quantile2 = df_temp['number_words'].quantile(.40)
number_words_quantile3 = df_temp['number_words'].quantile(.60)
number_words_quantile4 = df_temp['number_words'].quantile(.80)
df_temp['number_words_fifths'] = df_temp['number_words'].apply(lambda x: variable_levels(x, df_temp, 'number_words', number_words_quantile1, number_words_quantile2,
                                                                                  number_words_quantile3, number_words_quantile4))

df_temp[['id_session', 'number_words', 'number_words_fifths']].head(20)

In [ ]:
df_temp['number_words_fifths'].value_counts()

In [ ]:
df_temp.to_csv('before_regression_full_data_msg_perc_fifths_04_09.csv', index = 0)

# Customer's pleasurness with the service:

## Checking the distribution of the pleasurness variable:

In [ ]:
#Reading csv session.append and checking dist:
df_please = pd.read_csv("sessions_append.csv")
df_please["answer_csat"].value_counts()

### As can see most of customers did not answer (answer_csat = 0), lets present a Bar plot of the customers that did answer the survey:

In [ ]:
df_satisfaction = df_please[df_please['answer_csat'] > 0]
# df_satisfaction['answer_csat'].plot.bar(x = 'answer_csat', y = (df_satisfaction['answer_csat'].value_count()))
df_satisfaction['answer_csat'].value_counts().plot.bar(rot = 0)
# plt.xticks(np.arange(1,5.1))
plt.title('Customer pleasurness from service as answered')
round((df_satisfaction.shape[0]/session_append.shape[0]), 2)

### To identify the customers who answered the survey in the statistical models, lets merge df_please (session_append) with the regression table:

In [ ]:
# Reading the regression table:
df_reg = pd.read_csv("df_reg_no_small_shifts_06_03.csv", index_col= 0)

In [ ]:
# Slicing df_please to needed columns for the merge:
df_please_sliced = df_please[["id_session", "answer_csat"]]
# Merging:
df_merged = df_reg.merge(df_please_sliced, how= "left", on = 'id_session')
# Run to check the results of the merge:
df_merged[["id_session", "answer_csat"]].sort_values(by = "id_session").iloc[0:40], df_please[["id_session", "answer_csat"]].sort_values(by = "id_session").head(20)

In [ ]:
df_merged.shape[0] - df_reg.shape[0]

In [ ]:
df_merged = df_merged.drop_duplicates()

df_merged.shape

### Lets bar plot the pleasureness distribution in the regression table: 

In [ ]:
df_merged['answer_csat'].value_counts().plot.bar(rot = 0)
# plt.xticks(np.arange(1,5.1))
plt.title('Customer pleasurness from service in the regression table')

### Lets create a binary variable for customers who answered or not:

In [ ]:
# Creating a binary variable (customered answered or not the survey)
df_merged['binary_csat'] = df_merged['answer_csat'].apply(lambda x: 1 if x > 0 else 0)
# Run this to check the function:
df_merged[["id_session","answer_csat", "binary_csat"]].iloc[180:200]

In [ ]:
df_merged.groupby("chosen")[["binary_csat"]].count()

In [ ]:
df_merged[df_merged["answer_csat"] > 0].groupby("chosen")[["answer_csat"]].mean()

In [ ]:
df_merged.to_csv("df_reg_with_pleasurness.csv")

In [ ]:
# Creating a binary variable (customered answered or not the survey)
df_reg_transfer['binary_csat'] = df_reg_transfer['answer_csat'].apply(lambda x: 1 if x > 0 else 0)
# Run this to check the function:
df_reg_transfer[["id_session","answer_csat", "binary_csat"]].iloc[180:200]

In [ ]:
pd.crosstab(df_reg_transfer['binary_csat'], df_reg_transfer['chosen'])

# Try to describe more about the agent - shift length, call center information

##### agents_shifts df was obtained using sql server platform. In order to create shifts, we tracked each consequtive sequence of logout and then login, and mapped only those with a time interval of over 6 hours.

In [ ]:
agents_shifts = pd.read_excel('agents_shifts_16_3.xlsx')

In [ ]:
agents_shifts[['logout']]

In [ ]:
agents_shifts['logout'] = pd.to_datetime(agents_shifts['logout'], format = 'mixed')
# df_t = merged_session[merged_session[' agent_status'] == 5]
agents_shifts['logout'] = agents_shifts['logout'].dt.round('H').dt.hour

agents_shifts['logout'].hist(bins = 50)

In [ ]:
agents_shifts['log_in'] = pd.to_datetime(agents_shifts['log_in'], format = 'mixed')
# df_t = merged_session[merged_session[' agent_status'] == 5]
agents_shifts['log_in'] = agents_shifts['log_in'].dt.round('H').dt.hour

agents_shifts['log_in'].hist()

In [ ]:
agents_shifts['log_in'].hist(bins = 50)

In [ ]:
agents_shifts[['log_in', 'logout']].hist(bins = 50)

In [ ]:
agents_shifts.columns

In [ ]:
agents_shifts.groupby('t1_rep')['amount_shifts'].mean().hist(bins = 70)
plt.title('Amount of shifts for each agent')
plt.savefig('amount_shifts_agents.jpeg')

In [ ]:
agents_shifts['amount_shifts'].describe().round(2)

In [ ]:
hour_counts_login = agents_shifts['log_in'].value_counts(normalize=True).sort_index() * 100
hour_counts_logout = agents_shifts['logout'].value_counts(normalize=True).sort_index() * 100

In [ ]:
# Calculate percentage of each hour
# hour_counts = agents_shifts[['log_in', 'logout']].value_counts(normalize=True).sort_index() * 100

# Plot as a histogram with percentages
plt.figure(figsize=(8, 6))
plt.bar(hour_counts_login.index, hour_counts_login.values)
plt.xlabel('Hour')
plt.ylabel('Percentage (%)')
plt.title('Login - Percentage of Each Hour Out of All Hours')
plt.savefig('login_graph.png')

In [ ]:
# Calculate percentage of each hour
# hour_counts = agents_shifts[['log_in', 'logout']].value_counts(normalize=True).sort_index() * 100

# Plot as a histogram with percentages
plt.figure(figsize=(8, 6))
plt.bar(hour_counts_logout.index, hour_counts_logout.values)
plt.xlabel('Hour')
plt.ylabel('Percentage (%)')
plt.title('Logout - Percentage of Each Hour Out of All Hours')
plt.savefig('logout_graph.png')

## Engagement skill percentage of frequencies in the dataframe

In [ ]:
engagement_skill_perc = session_append['engagement_skill'].value_counts(normalize=True).sort_index() * 100
engagement_skill_perc

## Number of people in choice set - histogram

In [ ]:
def label_plot(plot1):
    for p in plot1.containers:
        plot1.bar_label(p, label_type='edge')
    return plot1

In [ ]:
plot = df_temp['choice_count'].value_counts().plot(kind = 'bar', rot = 0)
plt.xticks(np.arange(0, 6.1, 1))
plt.title('Number of customers in choice set')
label_plot(plot)
plt.savefig('people_choice_set.png')

## Creating a dataset where last messages of the conversation are filtered

### Checking how many choice sets with last messages exist

In [ ]:
df_temp = pd.read_csv('df_reg_with_quantiles_24_02.csv')

df_temp

In [ ]:
df_t = df_temp[df_temp['msg_perc'] == 1]
df_temp_not_last = df_temp[~df_temp['choice_set'].isin(df_t['choice_set'])]

df_temp_not_last.reset_index(inplace = True)

In [ ]:
df_temp_not_last['choice_count'].value_counts()

In [ ]:
df_temp.to_csv('df_reg_no_last_msgs.csv', index = 0)

In [ ]:
df_temp_not_last = pd.read_csv('df_reg_no_last_msgs_upd.csv')
df_temp_not_last['msg_perc'].max()

## Merging data for regression and agents data by number of shifts, so each choice set will show number of shifts the agent did during the month

In [ ]:
agents_shifts = pd.read_excel('agents_shifts_22_2.xlsx')

agents_shifts[agents_shifts['amount_shifts'] >=8]['t1_rep'].nunique() - agents_shifts['t1_rep'].nunique()

In [ ]:
df_temp[' id_rep']

In [ ]:
agents_shift_grouped = agents_shifts.groupby('t1_rep')['amount_shifts'].mean().reset_index()

agents_shift_grouped.rename(columns = {'t1_rep': ' id_rep'}, inplace = True)

agents_shift_grouped

In [ ]:
# Merging the dataframes after preparation:

# df_temp = df_temp.merge(agents_shift_grouped, on = ' id_rep', how = 'left')

df_t = df_temp[df_temp['amount_shifts'].notnull()]

df_t

In [ ]:
# df_t = df_t[df_t['amount_shifts'] >= 8]

df_temp.shape[0] - df_t.shape[0]

In [ ]:
df_t.to_csv('df_reg_no_small_shifts_06_03.csv')

In [ ]:
agents_shifts['minute_difference'].min()

## Descriptives of only the relevant login and logouts of defined shifts

In [ ]:
agents_shifts['logout'] = pd.to_datetime(agents_shifts['logout'], format = 'mixed')
# df_t = merged_session[merged_session[' agent_status'] == 5]
agents_shifts['logout'] = agents_shifts['logout'].dt.round('H').dt.hour

agents_shifts['logout'].hist(bins = 50)

In [ ]:
agents_shifts['log_in'] = pd.to_datetime(agents_shifts['log_in'], format = 'mixed')
# df_t = merged_session[merged_session[' agent_status'] == 5]
agents_shifts['log_in'] = agents_shifts['log_in'].dt.round('H').dt.hour

agents_shifts['log_in'].hist(bins = 50)

In [ ]:
hour_counts_login = agents_shifts['log_in'].value_counts().sort_index()
hour_counts_logout = agents_shifts['logout'].value_counts().sort_index()

In [ ]:
# Calculate percentage of each hour
# hour_counts = agents_shifts[['log_in', 'logout']].value_counts(normalize=True).sort_index() * 100

# Plot as a histogram with percentages
plt.figure(figsize=(8, 6))
plt.bar(hour_counts_login.index, hour_counts_login.values)
plt.xlabel('Hour')
plt.ylabel('Percentage (%)')
plt.title('Login - Percentage of Each Hour Out of All Hours')
plt.savefig('login_graph.png')

In [ ]:
# Calculate percentage of each hour
# hour_counts = agents_shifts[['log_in', 'logout']].value_counts(normalize=True).sort_index() * 100

# Plot as a histogram with percentages
plt.figure(figsize=(8, 6))
plt.bar(hour_counts_logout.index, hour_counts_logout.values)
plt.xlabel('Hour')
plt.ylabel('Percentage (%)')
plt.title('Logout - Percentage of Each Hour Out of All Hours')
plt.savefig('logout_graph.png')

## Deleting all the agents that belong to other engagement skills

In [ ]:
sessions_append = pd.read_csv('sessions_append.csv')
df_reg = pd.read_csv('df_reg_11_3_transfer_all_fifths.csv')
df_reg.drop(columns = ['X', 'Unnamed: 0'], inplace = True)
# Filtering session append according to relevant columns for the merge.
sessions_append = sessions_append[['id_session', 'engagement_skill', 'queue_time',
                                   'transfer_reason', 'chat_start_time', 'chat_end_time']]

##### Merging filtered session append and df_reg, using the how = left since we are interested in perserving columns in df_reg

In [ ]:
df_reg1 = df_reg.merge(sessions_append, how = 'left', on = 'id_session')

df_reg1.shape

##### Since there are the same session ids in the session append dataframe, that may reffer different days with the same people interacting, we are interested in merging according to the correct session ids according to the timing of the interaction. Therefore, we filter the end time of the event will be between the start and end time of the session (chat_start_time and chat_end_time).

In [ ]:
df_filtered = df_reg1[(df_reg1['end_time'] >= df_reg1['chat_start_time']) & 
                        (df_reg1['end_time'] <= df_reg1['chat_end_time'])]

df_filtered.shape

##### Deleting choice sets that include events with engagement skills that are not 1.

In [ ]:
df_t_irr_eng = df_filtered[df_filtered['engagement_skill'] != 1]
df_reg_no_engagement = df_filtered[~df_filtered['choice_set'].isin(df_t_irr_eng['choice_set'])]

df_reg_no_engagement.reset_index(inplace = True)

df_filtered['choice_set'].nunique() - df_reg_no_engagement['choice_set'].nunique()

##### Overall, 7104 choice sets were excluded, and around 16k events were deleted as a result

In [ ]:
# df_reg_no_engagement.drop(columns = 'index', inplace = True)
df_reg_no_engagement.to_csv('df_reg_engagement_del_no_small_shifts_18_3.csv')

## Attending a potential problem regarding duplication of sessions in the sessions_append original file.

##### We identified duplicated sessions in the session table (i.e, session append, merged sessions). Thus, we need to either erase these sessions before merging any kind of variables in the session level or erase them after merging to df_reg table. In any case, these sessions can not be a part of further analyses because they are problematic.

In [ ]:
merged_sessions[merged_sessions[' id_session'].duplicated()].sort_values(by = ' id_session').shape[0] / merged_sessions[' id_session'].nunique()

In [ ]:
ids_duplicated = merged_sessions[merged_sessions[' id_session'].duplicated()][' id_session']
filter1 = ids_duplicated.isin(df_reg['id_session'])
filter1.sum()
# merged_sessions = ids_duplicated[filter1]

# merged_sessions.isnull().sum()

In [ ]:
df_reg1[df_reg1.duplicated()]['id_session'].nunique()

In [ ]:
sessions_append.sort_values(by = 'id_session')

In [ ]:
sessions_append[sessions_append['id_session'].duplicated(keep = False)].sort_values(by = 'id_session').head(200)

In [ ]:
df_reg1.shape[0] - df_reg.shape[0]

In [ ]:
df_slice = df_reg.iloc[:20]

# df_slice.drop(columns = 'X', inplace = True)

df_slice1 = df_slice.merge(sessions_append[['id_session', 'engagement_skill', 'queue_time']], how = 'left', on = 'id_session')

In [ ]:
df_reg1.tail()

In [ ]:
df_slice.shape[0] - df_slice1.shape[0]

## Merging transfer variable into df_reg

In [ ]:
df_reg_transfer = df_merged.merge(sessions_append[['id_session', 'transfer_agent']], how = 'left', on = 'id_session')

df_reg_transfer

In [ ]:
df_reg_transfer.drop_duplicates(inplace = True)

df_reg_transfer.drop(columns = 'Unnamed: 0', inplace = True)

In [ ]:
df_reg_transfer['transfer_agent'].value_counts()

In [ ]:
1521/761957

In [ ]:
pd.crosstab(df_reg_transfer['transfer_agent'], df_reg_transfer['chosen'])

In [ ]:
df_reg_transfer.to_csv('df_reg_11_3.csv')

In [ ]:
# To check a connection between waiting time and termination type by system:

df_reg_transfer = pd.read_csv('df_reg_11_3_transfer_all_fifths.csv')

df_reg_transfer.columns

In [ ]:
df_reg_transfer.groupby('terminated_by_string')['waiting_time'].mean()

### Comparison between termination types by level of satisfaction

In [ ]:
df_answered = df_reg_transfer[df_reg_transfer['binary_csat'] == 1]
plot = df_answered.groupby('terminated_by_string')['answer_csat'].mean().round(2).plot(kind = 'bar', rot = 0,
                                                                                       title = 'Average of satisfaction for each termination type')
label_plot(plot)
plt.savefig('avg_satisfaction_termination.png')

In [ ]:
df_answered.columns

In [ ]:
df_answered[['answer_csat', 'msg_max']].corr()

In [ ]:
df = pd.read_csv('df_reg_11_3_transfer_all_fifths.csv')

df.columns

In [ ]:
df_transfers_updated = pd.read_csv('df_reg_11_3_transfer_all_fifths.csv')
df_all_fifths = pd.read_csv('df_reg_11_3_all_fifths_with_short_shifts.csv')

df_transfers_updated.shape[0] - df_all_fifths.shape[0]

### Working on new Dataframe for 8/4

##### Reading event level dataframe and creating a msg id column:

In [ ]:
df_event_merged = pd.read_csv("df_event_merged.csv", index_col=0)
df_event_merged["id_event"] = df_event_merged.index
df_event_merged.shape

##### Reading the session level data:

In [ ]:
session_df = pd.read_csv("sessions_append.csv")
session_df

##### Checking for duplicates in the session data

In [ ]:
print(f'There are {session_df["id_session"].duplicated().sum()} sessions duplicated')

##### These are the duplicated sessions (both duplicates are shwon through keep = false), I am getting rid of them as they are a very small percantage of the data:

In [ ]:
session_df_duplicated = session_df[session_df["id_session"].duplicated(keep= False)]
session_df_nodups = session_df[~ session_df["id_session"].isin(session_df_duplicated["id_session"])]

##### As can see, there are less sessions in the event level data than the session level data:

In [ ]:
df.rename(columns={' id_session': 'id_session'}, inplace = True)

df['id_session']

In [ ]:
df["id_session"].nunique() , session_df_nodups["id_session"].nunique(), session_df["id_session"].nunique()

In [ ]:
df.shape

### Merging the needed columns
##### Important note - There are sessions in the event level dont exist in the session level data. Thus, they get a none value after the merge: 

In [ ]:
df = df.rename(columns= {" id_session": "id_session"})
df_event_merged = df.merge(session_df_nodups[['id_session', 'queue_exit_time_date', 'chat_start_time_date', 'chat_end_time_date', 'queue_time', 'terminated_by']], how = 'left', on = 'id_session')
df_event_merged.isna().sum()

df_event_merged.shape

### Turning desired object columns into datetime type:
###### Important note -  some values (2 percent of the data, around 6500) get none becuase they dont have the format as all the other rows.   

In [ ]:
df_event_merged['queue_exit_time_date'] = pd.to_datetime(df_event_merged['queue_exit_time_date'], format= 'mixed', dayfirst= True)
df_event_merged['chat_start_time_date'] = pd.to_datetime(df_event_merged['chat_start_time_date'], format= 'mixed', dayfirst= True)
df_event_merged['chat_end_time_date'] = pd.to_datetime(df_event_merged['chat_end_time_date'], format= 'mixed', dayfirst= True)
df_event_merged[' end_date'] = pd.to_datetime(df_event_merged[' end_date'], format= 'mixed', dayfirst= True)
df_event_merged[' start_date'] = pd.to_datetime(df_event_merged[' start_date'], format= 'mixed', dayfirst= True)
df_event_merged.dtypes

In [ ]:
df_event_merged[["id_session" ,'queue_exit_time_date', 'chat_start_time_date', 'chat_end_time_date', ' end_date', ' start_date']].isnull().sum()

Filtering irrelevant events

In [ ]:
# Not needed in case of working with after dan dataset:
df_event_merged = df_event_merged[~((df_event_merged[' event_type'] == 7) | (df_event_merged[' event_type'] == 8) | (df_event_merged[' event_type'] == 9))]

In [ ]:
df_event_merged.shape

### Droping que rows

In [ ]:
df_event_merged = df_event_merged[df_event_merged[" event_type"] != 7]


df_event_merged.shape

### Creating desired columns:

##### Time since assignment

In [ ]:
df_event_merged['time_since_assignemnt'] = (df_event_merged[' end_date'] - df_event_merged['chat_start_time_date']).dt.seconds

In [ ]:
df_event_merged.shape

In [ ]:
df_event_merged[["id_session" ,'queue_exit_time_date', 'chat_start_time_date', 'chat_end_time_date', ' end_date', ' start_date', 'time_since_assignemnt']].head(20)

### Creating response time variable:

In [ ]:
df = df_event_merged.sort_values(by = ['id_session', ' end_date']).reset_index(drop = True)
df.head()

Response time = time (in seconds) that takes to respond to the last message, regardless of who sent the message.

In [ ]:
grouped = df.groupby('id_session')['time_since_assignemnt']

df['response_time'] = grouped.transform(lambda x: x - x.shift())

In [ ]:
df[" event_type"].value_counts()

In [ ]:
df.reset_index(inplace = True)

df.columns

Creating the if chat terminated variable

In [ ]:
df['chat_terminated'] = (df['id_session'] != df['id_session'].shift(-1)).astype(int)

df.head()

In [ ]:
df.to_csv('df_events_merged_with_response_&_terminated_by_10_06.csv', index= False)

09.06

---------------------------------------------------------

We want to create two columns: waiting time customer and waiting time agent:
- waiting time customer is the repsonse time of the customer TO the agent
- waiting time agent is the response time of the agent TO the customer

In [ ]:
df = pd.read_csv('df_events_merged_with_response_&_terminated_by_10_06.csv')

In [ ]:
def compute_waiting_time(group):
    return group.apply(
        lambda row: row['response_time'] if row[' event_type'] == 1 else None, axis=1
    ).fillna(group['response_time'].shift())

df['waiting_time_customer'] = df.groupby('id_session').apply(compute_waiting_time).reset_index(level=0, drop=True)

In [ ]:
df[['id_session', ' event_type_desc', 'waiting_time_customer', ' end_date']].iloc[500:600]

In [ ]:
def compute_waiting_time(group):
    return group.apply(
        lambda row: row['response_time'] if row[' event_type'] == 2 else None, axis=1
    ).fillna(group['response_time'].shift())

df['waiting_time_agent'] = df.groupby('id_session').apply(compute_waiting_time).reset_index(level=0, drop=True)

In [ ]:
df['waiting_time_agent'] = df.groupby('id_session').apply(
        lambda group: group.apply(lambda row: row['response_time'] if row[' event_type'] == 2 else None, axis=1).fillna(group['response_time'].shift())).reset_index(level=0, drop=True)

In [ ]:
df.rename(columns= {'waiting_time_agent': 'waiting_time_customer', 'waiting_time_customer': 'waiting_time_agent'}, inplace = True)

In [ ]:

df[[' end_date', " event_type_desc",'waiting_time_customer', 'waiting_time_agent']].head(30)

In [ ]:
def cat2(value):
    if value == 1:
        return "System"
    elif value == 2:
        return "Agent"
    elif value == 3:
        return "Customer"
    elif value == 4:
        return "Manager"
    else:
        None
df['terminated_by_string'] = df.apply(lambda row: cat2(row['terminated_by']), axis = 1)

In [ ]:
df.to_csv('df_events_merged_with_waiting_time_customer_&_agents_10_06.csv', index = 0)

In [ ]:
df = pd.read_csv('df_events_merged_with_waiting_time_customer_&_agents_13_04.csv')

df

In [ ]:
df_wanted = df[['id_session', ' id_rep', ' event_type_desc', 'time_since_assignemnt', 'response_time', 'chat_terminated', 'waiting_time_customer', 'waiting_time_agent', 'terminated_by_string']]

df_wanted.iloc[100:200]

In [ ]:
df_wanted_corr = df_wanted[['waiting_time_customer', 'waiting_time_agent']]


df_wanted_corr.corr()

In [ ]:
df.columns

In [ ]:
df_session = pd.read_csv('sessions_append_no_none_1_04.csv', index_col= 0)

In [ ]:
df_session.columns


In [ ]:
df.columns

In [ ]:
df_session['avg_words_agent'] = df_session['agent_number_words']/df_session['agent_lines']
df_session['avg_words_visitor'] = df_session['visitor_number_words']/df_session['visitor_lines']
df_session['avg_words'] = df_session['total_number_words']/df_session['total_lines']

In [ ]:
# GROUPING BY to aggregate waiting times for session level:
df_grouped = df.groupby("id_session")[["response_time",  "waiting_time_agent", "waiting_time_customer"]].mean()

df_grouped = df_grouped[df_grouped["response_time"].notna()]
df_grouped.reset_index(inplace = True)



In [ ]:
df_grouped.shape

In [ ]:
df_session_merged = df_session.merge(df_grouped, on = 'id_session', how = 'left')

In [ ]:
df_session_merged[df_session_merged['waiting_time_customer'].isna()]

In [ ]:
df[["id_session", " event_type_desc", " end_date", "response_time", "waiting_time_agent", "waiting_time_customer"]].head(50)



In [ ]:

df.rename(columns={'waiting_time_customer': 'waiting_time_agent', 'waiting_time_agent': 'waiting_time_customer'}, inplace= True)


In [ ]:
df

In [ ]:
df.groupby["id_session"][""]




df_for_merge = df[["id_session", "response_time", "waiting_time_agent", "waiting_time_customer"]]



In [ ]:
df_session_merged.to_csv('df_session_with_waiting_times_09_05.csv')

In [ ]:
df = pd.read_csv('df_session_with_waiting_times_09_05.csv', index_col= 0)

In [ ]:
df.head(100)

Engineering data for a regression on the session level, predicting termination types

In [ ]:
df['system_terminated_binary'] = df['terminated_by_string'].apply(lambda row: 1 if row == 'System' else 0)


df.head(30)

In [ ]:
df.columns


In [ ]:
sqldf('''select system_terminated_binary binary, 
                avg(average_sent) as average_sentiment, 
                avg(response_time) general_resp,
                avg(waiting_time_customer) agent_resp_time,
                avg(waiting_time_agent) customer_resp_time
            from df
            group by 1''')

In [ ]:
df[['id_session', 'answer_csat']].head(100)

In [ ]:
df_corr = df[df['answer_csat'] != 0]

In [ ]:
df_corr.corr(numeric_only= True)[['answer_csat']].sort_values(by = 'answer_csat')

In [ ]:
df_final.columns

In [ ]:
df_final.corr(numeric_only= True)[['system_terminated_binary']].sort_values(by = 'system_terminated_binary')

In [ ]:
df_corr[df_corr['id_visitor'].duplicated(keep = False)][['id_session', 'id_visitor', 'chat_start_time_date', 'average_sent', 'total_duration', 'answer_csat']].sort_values(['id_visitor', 'chat_start_time_date']).head(100)

In [ ]:
df[df['id_visitor'].duplicated(keep = False)][['id_session', 'id_visitor', 'chat_start_time_date', 'average_sent', 'total_duration']].sort_values(['id_visitor', 'chat_start_time_date']).head(100)

In [ ]:
sqldf('''
select count(id_visitor)
from df
where id_visitor = 200000001 
''')

In [ ]:
df_sql_q = sqldf(
'''
with cte_1 as (
select a.*,
        a.id_visitor first,
        b.id_visitor duplicated
from df a
left join df b
on a.id_visitor = b.id_visitor
AND a.id_session = b.id_session
order by a.id_visitor 
)

select *,
        row_number() over(partition by first) as occurances
from cte_1
''')

In [ ]:
df_sql_q[["occurances"]]

In [ ]:
df_final = sqldf('''
select *,
        case when lead(first) over(partition by first) = first then 1 else 0 end as binary_return 
from df_sql_q
''')


#

In [ ]:
df_sql_q.columns

In [ ]:
df_final[["chat_start_time_date", "id_session", "id_visitor", "occurances", "binary_return"]].sort_values(by = ['id_visitor', 'chat_start_time_date']).head(50)

In [ ]:
df_final.columns

In [ ]:
df_final.to_csv('session_level_with_binary_return_15_05.csv')

In [ ]:
df_final = pd.read_csv('session_level_with_binary_return_15_05.csv', index_col= 0)

df_final.head()

In [ ]:
len(df_final[(df_final['occurances'] > 1) & (df_final['answer_csat'] > 0)]['id_visitor'].unique())

## 26/5 - Nadav

- Predicting the return of customers (yes/no in 48 hours threshhold) by system closure (dummy of system)
- Create avg of sentiment for customer/agent
- Check correlation of return of customer and satisfaction.

In [ ]:
df_final['chat_start_time_date_dt'] = pd.to_datetime(df_final['chat_start_time_date'], format= 'mixed', errors = 'coerce')


In [ ]:
df_final =df_final.sort_values(by = ["id_visitor", "chat_start_time_date_dt"])

df_final[["chat_start_time_date_dt", "id_visitor"]].head(10)

In [ ]:
df_sql_q = sqldf(
'''
select *, 
        row_number() over(partition by id_visitor order by chat_start_time_date_dt) as occurances,
        lead()
from df_final
''')

df_sql_q.head(20)



In [ ]:
df_sql_q[["occurances"]]

In [ ]:
df_final = sqldf('''
select *,
        case when lead(id_visitor) over(partition by id_visitor) = id_visitor then 1 else 0 end as binary_return 
from df_sql_q
''')



df_final.head(50)
#

In [ ]:
df_filter = df_final[df_final['binary_return'] == 1]

df_returned = df_final[df_final['id_visitor'].isin(df_filter['id_visitor'])]


df_returned[["chat_start_time_date", "id_visitor", "occurances", "binary_return"]].head(10)

In [ ]:
df_returned[["chat_start_time_date", "chat_start_time_date_dt", "id_visitor", "occurances", "binary_return"]].head(10)
# Checking if there are problems:
df_returned[(df_returned['chat_start_time_date'].notna()) & (df_returned['chat_start_time_date_dt'].isna())]

In [ ]:
# sorting:
df_returned =df_returned.sort_values(by = ["id_visitor", "chat_start_time_date_dt"])
# Step 2: Calculate difference to next row
df_returned['chat_start_time_date_dt_next'] = df_returned.groupby("id_visitor")['chat_start_time_date_dt'].shift(-1)
df_returned['diff_to_next_hour'] = (df_returned['chat_start_time_date_dt_next'] - df_returned['chat_start_time_date_dt']).dt.total_seconds() / 3600



In [ ]:
df_returned_sliced = df_returned[["chat_start_time_date_dt", 'chat_start_time_date_dt_next', 'diff_to_next_hour', "id_visitor", "occurances", "binary_return"]]

df_returned_sliced.head(50)


In [ ]:
df_returned_sliced_48 = df_returned_sliced[df_returned_sliced["diff_to_next_hour"] <= 48]

df_returned_sliced_48.head(30)

In [ ]:
df_returned_sliced_48.shape[0]/df_returned_sliced.shape[0]

In [ ]:
df_returned_sliced = df_returned.sort_values(by = ["id_visitor", "chat_start_time_date_dt"])[["chat_start_time_date_dt", "id_visitor"]]

df_returned_sliced.head(20)


We identified a problem with the dates. Working on the dates

In [ ]:
session_df = pd.read_csv('df_final_binary_return_48_27_5.csv')

print(f'{session_df["chat_start_time_date"].min()}, {session_df["chat_start_time_date"].max()}') 

print(f'{session_df["chat_start_time_date_dt"].min()}, {session_df["chat_start_time_date_dt"].max()}') 



In [ ]:
# session_df['chat_start_time_date_dt_upd'] = pd.to_datetime(session_df['chat_start_time_date'], format='%d/%m/%Y %H:%M:%S')

session_df['chat_start_time_date_dt_upd'] = pd.to_datetime(session_df['chat_start_time_date'], format= 'mixed', dayfirst= True)
session_df[['chat_start_time_date', 'chat_start_time_date_dt', 'chat_start_time_date_dt_upd']].head(30)

In [ ]:
print(f'{session_df["chat_start_time_date_dt"].min()}, {session_df["chat_start_time_date_dt"].max()}') 

print(f'{session_df["chat_start_time_date_dt_upd"].min()}, {session_df["chat_start_time_date_dt_upd"].max()}') 

## 29/05 - Nadav

- Predicting the return of customers (yes/no in 48 hours threshhold) by system closure (dummy of system)
- Create avg of sentiment for customer/agent
- Check correlation of return of customer and satisfaction.

In [ ]:
df_final = pd.read_csv('df_final_binary_return_48_27_5.csv')

df_final['chat_start_time_date_dt_upd'] = pd.to_datetime(df_final['chat_start_time_date'], format= 'mixed', dayfirst= True)

df_final.head()


In [ ]:
df_final = df_final.sort_values(by = ["id_visitor", "chat_start_time_date_dt_upd"]).reset_index(drop= True)

df_final[["chat_start_time_date_dt_upd", "id_visitor"]].head(10)

In [ ]:
df_sql_q = sqldf(
'''
select *, 
        row_number() over(partition by id_visitor order by chat_start_time_date_dt_upd) as occurances_upd
from df_final
''')

df_sql_q.head(20)




In [ ]:
df_sql_q[["id_visitor", "occurances_upd", "chat_start_time_date_dt_upd"]].head(20)


In [ ]:
df_final = sqldf(
'''
select *,
        case when lead(id_visitor) over(partition by id_visitor) = id_visitor then 1 else 0 end as binary_return_upd
from df_sql_q
''')



df_final.head(50)


In [ ]:
df_final.drop(columns='occurances', inplace= True)
df_final['chat_start_time_date_dt_upd'] = pd.to_datetime(df_final['chat_start_time_date'], format= 'mixed', dayfirst= True)

df_final.head()

### Creating Variables:

In [ ]:
# sorting:
df_returned = df_final.sort_values(by = ["id_visitor", "chat_start_time_date_dt_upd"])
# Step 2: Calculate difference to next row
df_returned['chat_start_time_date_dt_next'] = df_returned.groupby("id_visitor")['chat_start_time_date_dt_upd'].shift(-1)
df_returned['diff_to_next_hour'] = (df_returned['chat_start_time_date_dt_next'] - df_returned['chat_start_time_date_dt_upd']).dt.total_seconds() / 3600

In [ ]:
df_broken = df_returned[(df_returned['binary_return_upd'] == 1) &  
                     (df_returned['diff_to_next_hour'].isna())][["chat_start_time_date_dt_upd", 'chat_start_time_date_dt_next',
                                                                                     'diff_to_next_hour', "id_visitor", "occurances_upd",
                                                                                       "binary_return_upd"]].sort_values(by = ['id_visitor', 'chat_start_time_date_dt_upd'])

df_broken.shape

In [ ]:
df_returned_48 = df_returned[df_returned["diff_to_next_hour"] <= 48]

df_returned_48.shape

In [ ]:
df_returned_48.shape[0]/df_returned.shape[0]

In [ ]:
df_returned_48.shape[0] / df_returned[df_returned["binary_return"] == 1].shape[0]

In [ ]:
df_returned_sliced = df_returned.sort_values(by = ["id_visitor", "chat_start_time_date_dt"])[["chat_start_time_date_dt", "id_visitor"]]

df_returned_sliced.head(20)


In [ ]:
df_filtered = df_final[df_final['occurances_upd'] <= 15]
df_filtered['occurances_upd'].hist(bins = 15)

In [ ]:
print(f'The percantage of unique customers that return to an agent after less than 48 hours out of all visitors : {round(df_returned_sliced_48["id_visitor"].nunique() / df_returned["id_visitor"].nunique(), 2)}')  

In [ ]:
print(f'The percantage of all sessions that after them the customer had to speak with an agent again in less than 48 hours out of allsession in which the customers have returned to an agent: {round(df_returned_sliced_48["id_visitor"].shape[0] / df_returned["id_visitor"].shape[0], 2)}')  

In [ ]:
print(f'The percantage of all sessions that after them the customer had to speak with an agent again in less than 48 hours out of all sessions: {round(df_returned_sliced_48["id_visitor"].shape[0] / df_final["id_visitor"].shape[0], 2)}')  

In [ ]:
print(f'The percantage of all sessions that after them the customer had to speak again with an agent: {round(df_returned[df_returned["binary_return"] == 1].shape[0] / df_final.shape[0], 2)}')

In [ ]:
df_returned["binary_48"] = df_returned["diff_to_next_hour"].apply(lambda x: 1 if x <= 48 else 0)


df_returned[["chat_start_time_date_dt_upd", 'chat_start_time_date_dt_next', 'diff_to_next_hour', "id_visitor", "occurances_upd", "binary_return_upd", "binary_48"]].iloc[50:70]

In [ ]:
df_returned.to_csv('session_df_29_05_dt_upd.csv', index= False)

---------------------------------------------------------------------------------------29.05----------------------------------------------------------------------

In [ ]:
df_filtered[["chat_start_time_date_dt", 'chat_start_time_date_dt_next', 'diff_to_next_hour', "id_visitor", "occurances", "binary_return", "binary_48"]].head(20)

In [ ]:
df_filtered_final = df_final[(df_final['occurances'] == 1)]

pd.crosstab(df_filtered_final['terminated_by_string'], df_filtered_final["binary_week"], margins = bool)

In [ ]:
df_final_diff = df_final[(df_final['diff_to_next_hour'] > 4000)] # & (df_final['diff_to_next_hour'] < 0)]

# df_final_diff['diff_to_next_hour'].hist(bins = 20)

df_final_diff[["chat_start_time_date_dt", 'chat_start_time_date_dt_next', 'diff_to_next_hour', "id_visitor", "occurances", "binary_return", "binary_week"]].head(50)

In [ ]:
df_sat["id_visitor"].nunique() / df_sat.shape[0]

### Looking at crosstabs of new created correct timespan:
### closure type and general returns:

In [ ]:
df_session = pd.read_csv('session_df_29_05_dt_upd.csv')

df_msg = pd.read_csv('merged_session_events.csv')

In [ ]:
pd.crosstab(df_session['terminated_by_string'], df_session["binary_return_upd"], margins = bool)

In [ ]:
pd.crosstab(df_session['terminated_by_string'], df_session["binary_48"], margins = bool)

In [ ]:
pd.crosstab(df_session['terminated_by_string'], df_session["binary_48"], normalize = "index")

In [ ]:
pd.crosstab(df_session['system_terminated_binary'], df_session["binary_48"], normalize = "index")

In [ ]:
df_sat = df_session[df_session["answer_csat"] != 0]

df_sat.groupby("binary_48")["answer_csat"].describe()

In [ ]:
df_sat = df_session[df_session["answer_csat"] != 0]

df_sat.groupby("binary_return_upd")["answer_csat"].describe()

In [ ]:
df_filtered = df_sat[(df_sat['occurances_upd'] == 1)]
#  & (df_sat['binary_return'] == 1)
df_filtered.groupby("binary_48")["answer_csat"].describe()

In [ ]:
df_filtered = df_sat[(df_sat['occurances_upd'] == 1)]
#  & (df_sat['binary_return'] == 1)
df_filtered.groupby("binary_return_upd")["answer_csat"].describe()

### Creation of binary week variable, if the customer have returned during a week time interval

In [ ]:
df_session["binary_week"] = df_session["diff_to_next_hour"].apply(lambda x: 1 if x <= 168 else 0)


df_session[["chat_start_time_date_dt", 'chat_start_time_date_dt_next', 'diff_to_next_hour', "id_visitor", "occurances_upd", "binary_48" ,"binary_return_upd", "binary_week"]].iloc[500:550]

In [ ]:
df_filtered = df_sat[(df_sat['occurances_upd'] == 1)]
#  & (df_sat['binary_return'] == 1)
df_filtered.groupby("binary_week")["answer_csat"].describe()

In [ ]:
df_session["diff_to_next_hour"].describe()

In [ ]:
df_sat.groupby("occurances_upd")["answer_csat"].mean()

In [ ]:
df_sat_dup = df_sat[df_sat['id_visitor'].duplicated(keep = False)].sort_values(by = ['id_visitor', "chat_start_time_date_dt_upd"])

##### Customers who do not return after the intial conversation are more satisfied, it is inconclusive yet to assume that the number of customers returns is associated with a decreasing level of satisfaction. However, data shows that customers which return is higher than the 5th time, are less satisfied with each return.

In [ ]:
df_filtered[df_filtered["binary_week"] == 1].shape[0] / df_filtered.shape[0]

### Creating sentiment variables for agents and customers, therefore returning to work with message level dataset

In [ ]:
df_msg = pd.read_csv('merged_session_events.csv')

In [ ]:
df_msg.rename(columns= {' id_session': 'id_session'}, inplace= True)

To distguish between repeating sessions when merging avg, mx, min sentiments we created a unique "id" for each session:

In [ ]:


df_session_new = sqldf(
'''
select *,
        row_number() over(partition by id_session order by chat_start_time_date_dt_upd) as row_num_session
from df
'''
)



In [ ]:
df_session_new['chat_start_time_date_dt_upd'] = pd.to_datetime(df_session_new['chat_start_time_date_dt_upd'], format= 'mixed', dayfirst= True)

In [ ]:
df_msg = df.rename(columns = {" event_type": "event_type", " id_session": "id_session", " sentiment": "sentiment"}

In [ ]:

df_msg_for_pivot = sqldf(
'''
select id_session, 
        event_type,
        avg(sentiment) as mean_sent,
        max(sentiment) as max_sent,
        min(sentiment) as min_sent
from df_msg
group by id_session, event_type
''')


In [ ]:
df_to_merge_quarters = df_msg_for_pivot.pivot(index='id_session', columns='event_type', values=["mean_sent", "max_sent", "min_sent"])

df_to_merge_quarters = df_to_merge_quarters.reset_index()
df_to_merge_quarters.reset_index(drop = True, inplace =True)

df_to_merge_quarters.head(20)

In [ ]:
df_to_merge_quarters.columns = ['_'.join(map(str, col)).strip() for col in df_to_merge_quarters.columns.values]
df_msg_for_merge = df_to_merge_quartersmerge_quarters.rename(columns = {"mean_sent_1": "avg_sent_visitor", 
                        "mean_sent_2": "avg_sent_agent", 
                        "max_sent_1": "max_sent_visitor", 
                        "max_sent_2": "max_sent_agent",
                        "min_sent_1": "min_sent_visitor",
                        "min_sent_2": "min_sent_agent"})

df_msg_for_merge.head(10)

In [ ]:
df_msg_for_merge.rename(columns = {"id_session_": "id_session"}, inplace = True)

In [ ]:
df_msg_for_merge.to_csv("sentiment_for_merge_10_06.csv", index = 0)

It seems that there is no documentation of sentiment level of messages by the agents

In [ ]:
df_session_merged = df_session.merge(df_msg_for_merge, on = 'id_session', how = 'left')

In [ ]:
df_session_merged.to_csv('df_session_with_sents_02_06.csv')

In [ ]:
df_session_merged = pd.read_csv('df_session_with_sents_02_06.csv', index_col= 0)

-------------------------------------------------------------------------------------------------------------------------

In [ ]:
df_events = pd.read_csv('merged_session_events.csv', index_col= 0)

In [ ]:
df_events = df_msg
df_events.columns

#### 8/6 - Creating fifths for mean waiting time in different phases of the conversation:

In [ ]:
df = pd.read_csv('df_events_merged_with_waiting_time_customer_&_agents_13_04.csv', index_col= 0)

df.drop(columns = ["Unnamed: 0"], inplace = True)

df.shape

In [ ]:
df['queue_exit_time_date'] = pd.to_datetime(df['queue_exit_time_date'], format= 'mixed', dayfirst= True)
df['chat_start_time_date'] = pd.to_datetime(df['chat_start_time_date'], format= 'mixed', dayfirst= True)
df['chat_end_time_date'] = pd.to_datetime(df['chat_end_time_date'], format= 'mixed', dayfirst= True)
df[' end_date'] = pd.to_datetime(df[' end_date'], format= 'mixed', dayfirst= True)
df[' start_date'] = pd.to_datetime(df[' start_date'], format= 'mixed', dayfirst= True)
df.dtypes

In [ ]:
df[' end_date'].max()

#### Checking the desired columns:

In [ ]:
df[['id_session', ' end_date', ' event_type_desc', 'waiting_time_customer', 'waiting_time_agent']].sort_values(['id_session', ' end_date']).head(50)

###### Renamaming colomns, as we mixed up them up:

In [ ]:
df.rename(columns= {'waiting_time_customer' : 'waiting_time_agent', 'waiting_time_agent': 'waiting_time_customer'}, inplace= True)

###### Creating the actual msg_perc column:

In [ ]:
df_events = df_events.sort_values(by = " end_time")
df_events['msg_count'] = df_events.groupby(by=['id_session'])[" end_time"].transform(lambda x: x.rank())
# Creating a column of total messages of conversation for each customer
df_temp = df_events[['msg_count', 'id_session']].groupby(by = 'id_session').max() 
df_temp['id_session'] = df_temp.index
df_temp.rename(columns = {"msg_count" : "msg_max"}, inplace = True)
df_temp.reset_index(drop = True, inplace = True)
df_perc = df_events.merge(df_temp, on = "id_session")
df_perc
df_perc['msg_perc'] = df_perc['msg_count'] / df_perc['msg_max']

df_perc['msg_perc'].describe()

##### Checking:

In [ ]:
df_perc[["id_session", " end_date", 'msg_count', 'msg_max', 'msg_perc']].sort_values(by= ["id_session", " end_date"]).head(50)

Creating a variable that filters to sessions that have at least 5 messages, for appropriate quarters

In [ ]:
df_perc["binary_msg_max_five"] = df_perc["msg_max"].apply(lambda x: 1 if x >= 5 else 0)


df_perc[["id_session", " end_date", 'msg_count', 'msg_max', 'msg_perc', "binary_msg_max_five"]].sort_values(by= ["id_session", " end_date"]).head(50)

In [ ]:
# Sroting the dataframe:
df_perc = df_perc.sort_values(by= ["id_session", " end_date"])


In [ ]:
df_perc.reset_index(drop = True, inplace = True)
df_perc.drop(columns = "Unnamed: 0", inplace = True)
df_perc.head()

##### Creating the quarters variable in order to group by for the waiting time mean:

In [ ]:
# The function that creates the partition to parts:

def variable_levels (row, quantile1, quantile2, quantile3):
    if row < quantile1:
        return 'quarter_1'
    elif (row >= quantile1) & (row < quantile2):
        return 'quarter_2'
    elif (row >= quantile2) & (row < quantile3):
        return 'quarter_3'
    elif (row >= quantile3):
        return 'quarter_4'
    else:
        None

In [ ]:
# Assigning variables and calling the function with apply and lambda:
msg_perc_quantile1 = 0.25
msg_perc_quantile2 = 0.5
msg_perc_quantile3 = 0.75
df_perc['msg_perc_quarters'] = df_perc['msg_perc'].apply(lambda x: variable_levels(x, msg_perc_quantile1, msg_perc_quantile2,
                                                                                  msg_perc_quantile3))

df_perc[['id_session', 'msg_perc', 'msg_perc_quarters', 'msg_count', 'msg_max']]

##### In order to get the mean waiting time for each quarter, we group by the session and quarter:

In [ ]:
df_grouped_wait = df_perc.groupby(["id_session", "msg_perc_quarters"])[["waiting_time_agent", 'waiting_time_customer', 'binary_msg_max_five']].mean()

df_grouped_wait.head(50)

In [ ]:
df_grouped_wait.reset_index(inplace = True)

In [ ]:
df_grouped_wait.columns

### Filtering for sessionthat have 5 or more events:

In [ ]:
df_grouped_wait_filt = df_grouped_wait[df_grouped_wait['binary_msg_max_five'] == 1]

In [ ]:
df_to_merge_quarters = df_grouped_wait_filt.pivot(index='id_session', columns='msg_perc_quarters', values=["waiting_time_agent", "waiting_time_customer"])
df_to_merge_quarters.head(10)


In [ ]:
df_to_merge_quarters.columns = ['_'.join(map(str, col)).strip() for col in df_to_merge_quarters.columns.values]
df_to_merge_quarters.head(20)

In [ ]:
df_to_merge_quarters.reset_index(inplace = True)

df_to_merge_quarters.head(20)

In [ ]:
df_to_merge_quarters.to_csv("waiting_time_avg_by_quarter_and_id_session_filtered_12_06.csv", index = 0)

In [ ]:
df_session = pd.read_csv('df_session_with_sents_02_06.csv', index_col= 0)

df_session.columns

In [ ]:
df_session_merged_12_06 = df_session.merge(df_to_merge_quarters, on = 'id_session', how = 'inner')

df_session_merged_12_06[["id_session", "chat_start_time_date_dt",'waiting_time_agent_quarter_1',
       'waiting_time_agent_quarter_2', 'waiting_time_agent_quarter_3',
       'waiting_time_agent_quarter_4', 'waiting_time_customer_quarter_1',
       'waiting_time_customer_quarter_2', 'waiting_time_customer_quarter_3',
       'waiting_time_customer_quarter_4']]

In [ ]:
df_session_merged_12_06[['waiting_time_agent_quarter_1', 'waiting_time_agent_quarter_2',
       'waiting_time_agent_quarter_3', 'waiting_time_agent_quarter_4',
       'waiting_time_customer_quarter_1', 'waiting_time_customer_quarter_2',
       'waiting_time_customer_quarter_3', 'waiting_time_customer_quarter_4']].describe()

In [ ]:
df_session_merged_12_06.to_csv('session_merged_5_msg_with_quarters_12_06.csv', index= False)

In [ ]:
df_session_merged[['id_session', 'total_lines', ""]].head(50)

In [ ]:
df_session_merged_10_06[['waiting_time_agent_quarter_1',
       'waiting_time_agent_quarter_2', 'waiting_time_agent_quarter_3',
       'waiting_time_agent_quarter_4']].isna().sum()

In [ ]:
df_perc[df_perc["id_session"] == 100160904]

In [ ]:
df_hist = df_perc[df_perc['msg_max'] <= 50]
df_hist.groupby('id_session')['msg_max'].count().hist(bins = 70)

Regression in R deleted half of observations. Analyzing how many NaNs do we have in each variable

In [ ]:
df_session_merged_12_06[['waiting_time_agent_quarter_1', 'waiting_time_agent_quarter_2',
       'waiting_time_agent_quarter_3', 'waiting_time_agent_quarter_4',
       'waiting_time_customer_quarter_1', 'waiting_time_customer_quarter_2',
       'waiting_time_customer_quarter_3', 'waiting_time_customer_quarter_4']].isna().sum()

In [ ]:
df_to_merge_quarters[df_to_merge_quarters['waiting_time_agent_quarter_1'].isna()].head(50)

In [ ]:
df_perc[df_perc['id_session'] == 100000010][[' end_date', 'waiting_time_customer', 'waiting_time_agent', 'msg_perc_quarters']]

In [ ]:
df_session_merged_12_06['answer_csat'].value_counts()

In [ ]:
df_session_csat = df_session_merged_12_06[df_session_merged_12_06['answer_csat'] != 0]

In [ ]:
df_session_csat.to_csv('session_merged_csat_12_06.csv', index= False)

In [ ]:
df_session_csat[['waiting_time_agent_quarter_1', 'waiting_time_agent_quarter_2',
       'waiting_time_agent_quarter_3', 'waiting_time_agent_quarter_4',
       'waiting_time_customer_quarter_1', 'waiting_time_customer_quarter_2',
       'waiting_time_customer_quarter_3', 'waiting_time_customer_quarter_4']].isna().sum()

In [ ]:
df_session_csat['answer_csat'].describe()

In [ ]:
df_1 = pd.read_csv('before_analysis_0to1m.csv', index_col = 0)
df_2 = pd.read_csv('before_analysis_2milion.csv', index_col = 0)
df_3 = pd.read_csv('all_filtered_events_2m-end_before_analysis.csv', index_col = False)

In [ ]:
df_1.drop(columns={'index'}, inplace = True)
df_2.drop(columns={'index'}, inplace = True)

In [ ]:
df_1.rename(columns = {'wating_time': 'waiting_time'}, inplace = True)
df_2.rename(columns = {'wating_time': 'waiting_time'}, inplace = True)

In [ ]:
df_1['choice_set'].max()

In [ ]:
df_2['choice_set'] = df_2['choice_set'] + df_1['choice_set'].max() + 1

df_2.head(50)

In [ ]:
df_2['choice_set'].max()

In [ ]:
df_3['choice_set'] = df_3['choice_set'] + df_2['choice_set'].max() + 1

df_3.head(50)

In [ ]:
df_all = pd.concat([df_1, df_2, df_3])
df_all.reset_index(drop = True, inplace = True)

df_all.shape

In [ ]:
df_all = df_all.sort_values(by = ['choice_set', ' end_date'])

In [ ]:
df_all.head(50)

In [ ]:
df_all.to_csv('all_filtered_events_all_data_18_06.csv', index = 0)

In [ ]:
df_all = pd.read_csv('all_filtered_events_all_data_18_06.csv')

In [ ]:
# dummy of people who got chosen from others in a choice set, that is those who got 1 when others got 0
# same but for unchosen, that is those who got 0 but someone else got 1
# dummy of times he was alone in the choice set
# ignored therefore got 0 towards the end of the session

In [ ]:
# Creating id  for each message

df_all['id'] = df_all.index
# Filtering all unique sessions:
df_dup = df_all[df_all["choice_set"].duplicated(keep = False)].sort_values(by = [" id_rep", " end_date"])
# Filtering only chosen sessions:
df_chosen = df_dup[df_dup["chosen"] == 1].sort_values(by = [" id_rep", " end_date"])

df_chosen.reset_index(drop = True, inplace = True)

In [ ]:
df_chosen['chosen_from_others_dummy'] = 1
df_chosen_sliced = df_chosen[['id', 'chosen_from_others_dummy']]

df_all_merged = df_all.merge(df_chosen_sliced, how = 'left', on = 'id')

df_all_merged['chosen_from_others_dummy'] = df_all_merged['chosen_from_others_dummy'].apply(lambda x: 0 if pd.isna(x) else 1)
df_all_merged[[' end_date', ' id_session', 'choice_set', 'chosen', 'chosen_from_others_dummy']].head(50)

In [ ]:
# Filtering only chosen sessions:
df_unchosen = df_dup[df_dup["chosen"] == 0].sort_values(by = [" id_rep", " end_date"])

df_unchosen.reset_index(drop = True, inplace = True)

In [ ]:
df_unchosen['not_chosen_from_others_dummy'] = 1
df_unchosen_sliced = df_unchosen[['id', 'not_chosen_from_others_dummy']]

df_all_merged_2 = df_all_merged.merge(df_unchosen_sliced, how = 'left', on = 'id')

df_all_merged_2['not_chosen_from_others_dummy'] = df_all_merged_2['not_chosen_from_others_dummy'].apply(lambda x: 0 if pd.isna(x) else 1)
df_all_merged_2[[' end_date', ' id_session', 'choice_set', 'chosen', 'not_chosen_from_others_dummy', 'chosen_from_others_dummy']].head(50)

In [ ]:
df_all_merged_2.to_csv('df_all_with_chosen_and_not_chosen_from_others_19_06.csv', index = 0)

In [ ]:
df = pd.read_csv("df_all_with_chosen_and_not_chosen_from_others_19_06.csv")

df.head(20)

In [ ]:
# Filtering all non-unique sessions (I want only unique):
df_not_dup = df[~(df["choice_set"].duplicated(keep = False))].sort_values(by = [" id_rep", " end_date"])

df_not_dup.head(20)


4.5 % are not chosen and are alone in the choice-set!

In [ ]:
df_not_dup_chosen = df_not_dup[df_not_dup["chosen"] == 1]
df_not_dup_chosen['chosen_alone'] = 1
df_not_dup_chosen_sliced = df_not_dup_chosen[['id', 'chosen_alone']]
df_all_merged_2 = df.merge(df_not_dup_chosen_sliced, how = 'left', on = 'id')
df_all_merged_2['chosen_alone'] = df_all_merged_2['chosen_alone'].apply(lambda x: 0 if pd.isna(x) else 1)
df_all_merged_2[[' end_date', ' id_session', 'choice_set', 'chosen', 'not_chosen_from_others_dummy', 'chosen_from_others_dummy', "chosen_alone"]].head(50)


In [ ]:
df_chosen = df[df["chosen"] == 1] 
df_last_chosen = df.groupby(" id_session")[["id"]].max().reset_index()

In [ ]:
'''
1. create a new column in this df of last_chosen = 1
2. merge this to original df
3. if NA give 0
4. give 1 to each messege of id session that was sent after last_chosen == 1. 
'''



Handeling non-duplicates and chosen = 0 sessions - Nadav 24/6:

In [ ]:
df = pd.read_csv("df_all_with_chosen_and_not_chosen_from_others_19_06.csv")


In [ ]:
df.head(30)

In [ ]:
# Filtering all non-unique sessions (I want only unique):
df_not_dup = df[~(df["choice_set"].duplicated(keep = False))].sort_values(by = [" id_rep", " end_date"])
df_not_dup_unchosen = df_not_dup[df_not_dup["chosen"] == 0]
df_not_dup_unchosen[[" id_session", " id_rep", " end_date", "choice_set", "chosen"]].head(20)



In [ ]:
df_after_dan = pd.read_csv("merged_df_after_dan_6m.csv", index_col = 0)

df_after_dan = df_after_dan.sort_values(by = [" id_rep", " end_time"]) 

In [ ]:
df_after_dan[df_after_dan[" id_rep"] == 30000002][[" id_session" ," end_date", " event_type_desc"]].iloc[400:500]

In [ ]:
df_after_dan[df_after_dan[" id_session"] == 100029275][[" id_session" ," end_date", " event_type_desc"]]

#### Nadav 7/4/25:


In [ ]:
df_session = pd.read_csv("session_level_with_all_columns_24_06.csv")


df_session.head(20)

In [ ]:

df_session_filt = df_session[["id_session", "chat_end_time_date", 'chosen_from_others', 
            'not_chosen_from_others_dummy', 'chosen_alone',  'after_last_and_not_chosen', 
             "from_others", "from_others_single_chosen", "from_others_no_single_chosen_with_last_lag",
            "from_others_single_chosen_with_last_lag"]]

df_session_filt[df_session_filt["after_last_and_not_chosen"] > 0]

In [ ]:
df_all[df_all[" id_session"] == 100075498]



In [ ]:
df_all = pd.read_csv("df_all_24_06_with_after_last_and_not_chosen.csv")



df_all.columns

In [ ]:
df_session.to_csv("df_session_all_variables_4_7.csv")

In [ ]:
df_session = pd.read_csv('df_session_all_variables_4_7.csv')

Creating prioritization variables

In [ ]:
""" 
from others + from others no single chosen with last lag = excluding only when the participant is alone in the choice set

from_others_single_chosen_with_last_lag = all times that participant was chosen out from all times he appeared in a choice set




"""

In [ ]:
# Percentage out of customer being chosen out of all times hes in choice set with others
df_session["from_others"] = df_session["chosen_from_others"] / (df_session["chosen_from_others"] + df_session["not_chosen_from_others_dummy"]) 
# Percentage of customer being chosen out of all the times he is in a choice set, with others AND alone (without cases of being alone in choice set and NOT being chosen, talk with galit)
df_session["from_others_single_chosen"] = (df_session["chosen_from_others"] + df_session["chosen_alone"]) / (df_session["chosen_from_others"] + df_session["not_chosen_from_others_dummy"] + df_session["chosen_alone"])  
df_session["from_others_no_single_chosen_with_last_lag"] = df_session["chosen_from_others"]  / (df_session["chosen_from_others"] + df_session["not_chosen_from_others_dummy"] + df_session["after_last_and_not_chosen"])  
df_session["from_others_single_chosen_with_last_lag"] = (df_session["chosen_from_others"] + df_session["chosen_alone"] ) / (df_session["chosen_from_others"] + df_session["not_chosen_from_others_dummy"] + df_session["after_last_and_not_chosen"] + df_session["chosen_alone"])  


In [ ]:
df_temp = df_session[["id_session", "chat_end_time_date", 'chosen_from_others', 
            'not_chosen_from_others_dummy', 'chosen_alone',  'after_last_and_not_chosen', 
             "from_others", "from_others_single_chosen", "from_others_no_single_chosen_with_last_lag",
            "from_others_single_chosen_with_last_lag"]]

In [ ]:
df_temp[df_temp['from_others_single_chosen'] != df_temp['from_others_single_chosen_with_last_lag']]

Creating Excluding last lag variables

In [ ]:
# Filtering all non-unique sessions (I want only unique):
df_not_dup = df_all[~(df_all["choice_set"].duplicated(keep = False))].sort_values(by = [" id_rep", " end_date"])

df_not_dup.head(20)

In [ ]:
df_not_chosen_not_dup = df_not_dup[df_not_dup["chosen"] == 0]
df_not_chosen_not_dup['not_chosen_alone'] = 1
df_not_chosen_not_dup_sliced = df_not_chosen_not_dup[['id', 'not_chosen_alone']]
df_all_merged_2 = df_all.merge(df_not_chosen_not_dup_sliced, how = 'left', on = 'id')
df_all_merged_2['not_chosen_alone'] = df_all_merged_2['not_chosen_alone'].apply(lambda x: 0 if pd.isna(x) else 1)
df_all_merged_2[[' end_date', ' id_session', 'choice_set', 'chosen', 'not_chosen_from_others_dummy', 'chosen_from_others_dummy', "not_chosen_alone"]].head(50)

In [ ]:
df_all_merged_2[df_all_merged_2['not_chosen_alone'] == 1]['not_chosen_alone']

In [ ]:
column = df_all_merged_2.groupby(' id_session')[['not_chosen_alone']].sum()
column.reset_index(inplace= True)

In [ ]:
df_session.columns

In [ ]:
column.columns

In [ ]:
#column.rename(columns= {' id_session': 'id_session'}, inplace= True)
df_session = df_session.merge(column, on = 'id_session', how = 'left')

In [ ]:
df_session['not_chosen_alone'].hist()

In [ ]:
df_temp = df_session[["id_session", "chat_end_time_date", 'chosen_from_others', 
            'not_chosen_from_others_dummy', 'chosen_alone', 'not_chosen_alone',  'after_last_and_not_chosen', 
             "from_others", "from_others_single_chosen", "from_others_no_single_chosen_with_last_lag",
            "from_others_single_chosen_with_last_lag"]]

df_temp.iloc[180:190]


In [ ]:
df_session["prioritization_without_last_lag_exclude_alone"] = df_session["chosen_from_others"]  / (df_session["chosen_from_others"] + (df_session["not_chosen_from_others_dummy"] + df_session["not_chosen_alone"] - df_session['after_last_and_not_chosen']))



df_session["prioritization_without_last_lag_include_alone"] = (df_session["chosen_from_others"] + df_session['chosen_alone']) / (df_session["chosen_from_others"] + df_session['chosen_alone'] + (df_session["not_chosen_from_others_dummy"] + df_session["not_chosen_alone"] - df_session['after_last_and_not_chosen']))

In [ ]:
df_temp = df_session[["id_session", "chat_end_time_date", 'chosen_from_others', 
            'not_chosen_from_others_dummy', 'chosen_alone', 'not_chosen_alone',  'after_last_and_not_chosen', 
             "from_others", "from_others_single_chosen", "from_others_no_single_chosen_with_last_lag",
            "from_others_single_chosen_with_last_lag", 'prioritization_without_last_lag_exclude_alone', 'prioritization_without_last_lag_include_alone']]

df_temp.iloc[180:190]

In [ ]:
df_session.columns

In [ ]:
df_session.drop(columns= ['from_others', 'from_others_single_chosen'], inplace= True)


df_session.columns

In [ ]:
df_session.to_csv('df_session_with_prioritization_vars_07_07.csv', index= 0)

Investigation regarding missing sessions from big code chunk

In [ ]:
session_events = pd.read_csv('merged_session_events.csv')

In [ ]:
merged_df = pd.read_csv('merged_df_after_dan_6m.csv', index_col = 0)

In [ ]:
session_events[' id_session'].nunique()

In [ ]:
merged_df[' id_session'].nunique()

In [ ]:
df_all = pd.read_csv('df_all_24_06_with_after_last_and_not_chosen.csv')

df_all[' id_session'].nunique()

In [ ]:
df_choice_sets[' id_session'].nunique()

In [ ]:
df_sessions_deleted = session_events[~(session_events[' id_session'].isin(df_all[' id_session']))]

In [ ]:
df_sessions_deleted[' id_session'].unique()[0:20]

In [ ]:
session_events[session_events[' id_session'] == 100007392][[' id_visitor', ' id_session', ' id_rep', ' start_date', ' start_time',
       ' end_date', ' end_time', ' duration', ' event_type',
       ' event_type_desc', ' sentiment', ' number_words']]

In [ ]:
merged_df[merged_df[' id_session'] == 100007392][[' id_visitor', ' id_session', ' id_rep', ' start_date', ' start_time',
       ' end_date', ' end_time', ' duration', ' event_type',
       ' event_type_desc', ' sentiment', ' number_words']]

In [ ]:
merged_df[merged_df[' id_rep'] == 30000400][[' id_visitor', ' id_session', ' id_rep', ' start_date', ' start_time',
       ' end_date', ' end_time', ' duration', ' event_type',
       ' event_type_desc', ' sentiment', ' number_words']].sort_values(by = ' start_time').head(100)

In [ ]:
session_events[session_events[' id_rep'] == 30000473][[' id_visitor', ' id_session', ' id_rep', ' start_date', ' start_time',
       ' end_date', ' end_time', ' duration', ' event_type',
       ' event_type_desc', ' sentiment', ' number_words']].head(100)

In [ ]:
df_session = pd.read_csv('merged_session.csv')

df_session

In [ ]:
df_session.columns

In [ ]:
df_session[df_session[' id_rep'] == 30001042]

In [ ]:
df_session[(df_session[' chat_end_time'] > 1493599911) & (df_session[' chat_start_time'] < 1493599911) & (df_session[' id_rep'] == 30001042)][' id_session'].unique

Creating variable of sentiment of first message for each session

In [ ]:
df_session = pd.read_csv('df_session_with_prioritization_vars_07_07.csv')

In [ ]:
df_session.columns

In [ ]:
first_msg_df = session_events.groupby([' id_session']).first().reset_index()

In [ ]:
first_msg_df = first_msg_df[[' id_session', ' number_words', ' sentiment', ' event_type', ' event_type_desc']]

In [ ]:
df_session.columns

In [ ]:
first_msg_df.rename(columns= {' id_session': 'id_session', ' number_words': 'number_words_first_msg', ' sentiment': 'sentiment_first_msg', ' event_type': 'event_type_first_msg', ' event_type_desc': 'event_type_desc_first_msg'}, inplace= True)

first_msg_df

In [ ]:
first_msg_df.columns

In [ ]:
df_session_with_first = df_session.merge(first_msg_df, on = 'id_session', how = 'left')


df_session_with_first

In [ ]:
df_session_with_first.to_csv('df_session_with_first_msg_upd_15_07.csv', index= 0)

19.07 checking if chat_start_time is accurate to first msg that appears in msg level dataset

In [ ]:
session_events = session_events_1

In [ ]:
session_events = session_events.sort_values(by = [' id_session', ' end_time'])

In [ ]:
session_events.shape

In [ ]:
df_session[df_session[' id_session'] == 100000006]

Create two variables: Waiting time from 'Chat_start_date' and waiting time from 'queue_exit_date' (both session level)

In [ ]:
session_events[' id_session'].nunique()

In [ ]:
event_type_7 = session_events[session_events[' event_type'] == 7]
session_events['queue_exit_date'] = event_type_7[' end_date']

In [ ]:
session_events[(session_events[' id_session'].isin(event_type_7[' id_session']))]

In [ ]:
session_events[session_events[' id_session'] == 100000017]

In [ ]:
session_events.shape

In [ ]:
id_rep_1 = session_events[session_events[' id_rep'] == 1]

grouped = id_rep_1.groupby(' id_session')[[' id_session']].count()
grouped

In [ ]:
id_rep_1[" event_type"].value_counts()

Trying to move the value of queue_exit_date to the first row of id_rep = 1

In [ ]:
# נשלוף את זמן event_type == 7 לכל סשן
event7_times = id_rep_1[id_rep_1[' event_type'] == 7].groupby(' id_session')[' end_date'].first().reset_index()
event7_times

# event7_times = event7_times.rename(columns={' end_date': 'assignment_date'})




# # נאתר את שורות event_type == 1 המוקדמות ביותר בכל סשן
# first_event1 = id_rep_1[id_rep_1[' event_type'] == 1].sort_values(' end_time').groupby(' id_session').first().reset_index()

# # נוסיף את זמן event_type 7 לשורות האלה
# first_event1 = first_event1.merge(event7_times, on=' id_session', how='left')

# # עכשיו נחבר את המידע הזה חזרה ל־id_rep_1 המקורי
# # נוסיף עמודת event7_time לכל שורה ב־id_rep_1, אבל רק לשורות של האירוע הראשון מסוג 1
# session_events_temp = session_events.merge(
#     first_event1[[' id_session', ' end_date', 'assignment_date']],
#     on=[' id_session', ' end_date'], how='left')

In [ ]:
session_events

In [ ]:
session_events[session_events[' id_session'] == 100000002]

In [ ]:
session_events['event_id'] = session_events.index

Deleting all event_type 7, as we took their relevent date_times that indicate the time each customer got assigned to an agent

In [ ]:
session_events = session_events[~(session_events[' event_type'] == 7)]

In [ ]:
session_events[' event_type'].value_counts()

Creating dummy for first msg of the session

In [ ]:
session_events['first_msg_dummy'] = session_events[' id_rep'].apply(lambda x: 1 if x == 1 else 0)


session_events

Now we transform the id_rep = 1 to the agent's id that are assigned right after

In [ ]:
# נשלוף את זמן event_type == 7 לכל סשן
id_reps = id_rep_1[id_rep_1[' event_type'] == 7].groupby(' id_session')[' end_date'].first().reset_index()
id_reps = id_reps.rename(columns={' end_date': 'assignment_date'})

# נאתר את שורות event_type == 1 המוקדמות ביותר בכל סשן
first_event1 = id_rep_1[id_rep_1[' event_type'] == 1].sort_values(' end_time').groupby(' id_session').first().reset_index()

# נוסיף את זמן event_type 7 לשורות האלה
first_event1 = first_event1.merge(event7_times, on=' id_session', how='left')

# עכשיו נחבר את המידע הזה חזרה ל־id_rep_1 המקורי
# נוסיף עמודת event7_time לכל שורה ב־id_rep_1, אבל רק לשורות של האירוע הראשון מסוג 1
session_events = session_events.merge(
    first_event1[[' id_session', ' end_date', 'assignment_date']],
    on=[' id_session', ' end_date'], how='left')

In [ ]:
id_reps = session_events[~(session_events[' id_rep'] == 1)].groupby(' id_session')[[ ' id_rep']].first().reset_index()


id_reps

In [ ]:
session_events_merged = .merge(id_reps, on = ' id_session', how = 'left')session_events

In [ ]:
session_events

In [ ]:
session_events_merged

In [ ]:
session_events_merged.drop(columns= ' id_rep_x', inplace= True)

session_events_merged.rename(columns= {' id_rep_y': ' id_rep'}, inplace= True)

session_events_merged

In [ ]:
session_events_merged[' id_rep'] = session_events_merged[" id_rep"].astype('Int64')

In [ ]:
session_events_merged

In [ ]:
session_events_merged.to_csv('raw_session_events_before_events_8_9_19_07.csv', index= 0)

Now we take this to the original code way above

# THRESHOLD

In [ ]:
df_session = pd.read_csv('df_session_csat.csv')

df_session

In [ ]:
list = list(df_session.columns)

In [ ]:
print(list)

In [ ]:
df_session_new = df_session[['id_session', 'chat_start_time', 'queue_exit_time', 'chat_end_time', 'queue_time', 'transfer_agent', 'transfer_reason', 'answer_csat', 'chat_sentiment','id_rep','visitor_duration', 'agent_duration', 'total_duration', 'visitor_number_words', 'agent_number_words', 'total_number_words', 'visitor_number_chars', 'agent_number_chars', 'total_number_chars', 'visitor_lines', 'agent_lines', 'total_lines', 'average_sent', 'min_sent', 'max_sent', 'n_sent', 'n_sent_pos', 'n_sent_neg', 'first_sent', 'last_sent', 'last_line_time', 'closer_time', 'id_rep_code', 'cust_prob_waiting','terminated_by', 'chat_start_time_date', 'chat_end_time_date', 'terminated_by_string', 'avg_words_agent', 'avg_words_visitor', 'avg_words', 'waiting_time_agent', 'waiting_time_customer', 'system_terminated_binary', 'binary_48', 'chat_start_time_date_dt_upd', 'occurances_upd', 'binary_return_upd', 'binary_week', 'avg_sent_visitor', 'avg_sent_agent', 'max_sent_visitor', 'max_sent_agent', 'min_sent_visitor', 'min_sent_agent', 'waiting_time_agent_quarter_1', 'waiting_time_agent_quarter_2', 'waiting_time_agent_quarter_3', 'waiting_time_agent_quarter_4', 'waiting_time_customer_quarter_1', 'waiting_time_customer_quarter_2', 'waiting_time_customer_quarter_3', 'waiting_time_customer_quarter_4', 'chosen_from_others', 'not_chosen_from_others_dummy', 'chosen_alone', 'after_last_and_not_chosen']]

In [ ]:
df_session_new

In [ ]:
df_session_new.to_csv('df_session_for_test_21_8.csv', index = 0)

In [ ]:
del list

In [ ]:
df_session_new = pd.read_csv('df_session_for_test_21_8.csv')

In [ ]:
records = df_session_new.to_dict("records")
csat_scores = list(map(lambda x: x["answer_csat"], records))
print(csat_scores[:10])

In [ ]:
df = pd.read_csv('before_regression_full_data_msg_perc_fifths_04_09.csv')

In [ ]:
# --- Step 1: get only the chosen customer per choice_set ---
chosen_per_set = (
    df.loc[df["chosen"] == 1, ["id_rep", "choice_set", "end_time", "id_session"]]
    .sort_values(["id_rep", "end_time"])  # chronological order within each rep
)

# --- Step 2: keep only one chosen per (id_rep, choice_set) ---
# If there are duplicates, keep the last one by end_time
chosen_per_set = (
    chosen_per_set
    .drop_duplicates(["id_rep", "choice_set"], keep="last")
)

# --- Step 3: compute the previous chosen customer per agent ---
chosen_per_set["prev_customer"] = (
    chosen_per_set.groupby("id_rep")["id_session"].shift(1)
)

# --- Step 4: create mapping of prev_customer for each set ---
mapping = chosen_per_set[["id_rep", "choice_set", "prev_customer"]]

# --- Step 5: merge this mapping onto ALL rows in df ---
df = df.merge(mapping, on=["id_rep", "choice_set"], how="left", validate="many_to_one")

# --- Step 6: stickiness flag ---
# A row gets 1 iff its session == the customer chosen in the last choice set
df["stickiness"] = (df["id_session"] == df["prev_customer"]).astype(int)
df["stickiness"] = df["stickiness"].fillna(0).astype(int)

# --- Optional clean-up ---
df = df.drop(columns=["prev_customer"])

In [ ]:
df.to_csv('before_regression_full_data_msg_perc_fifths_10_09.csv', index = 0)

In [ ]:
df_exploded = pd.read_csv('df_exploded_all_data_24_08.csv')

df_exploded

In [ ]:
df_exploded.sort_values(by = ['id_rep', 'time'], inplace = True)

df_exploded[['time', 'id_rep']].head(50)

In [ ]:
df_exploded['time'].unique()

In [ ]:
df_exploded[~df_exploded['time'].duplicated()]

-------------------------------

In [5]:
session_events_merged = pd.read_csv('raw_session_events_before_events_8_9_19_07.csv')

C:\Users\Tamir\AppData\Local\Temp\ipykernel_30984\3828195404.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  session_events_merged = pd.read_csv('raw_session_events_before_events_8_9_19_07.csv')


In [4]:
df_choice_set = pd.read_csv('all_filtered_events_all_data_26_08.csv')

df_choice_set


C:\Users\Tamir\AppData\Local\Temp\ipykernel_30984\257646500.py:1: DtypeWarning: Columns (0,1,2,3,5,7,8,9,11,12,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_choice_set = pd.read_csv('all_filtered_events_all_data_26_08.csv')


,event_id,id_visitor,id_session,id_rep,start_date,start_time,end_date,end_time,duration,event_type,event_type_desc,sentiment,number_words,assignment_date,first_msg_dummy,choice_set,chosen,waiting_time,workload
0,5969556,200140251,100247275,30000002.0,01/05/2017 00:22:57,1493598177,01/05/2017 00:22:57,1493598177,0,1,visitor_line,0,36,01/05/2017 00:24:04,1,0,1,81,22
1,5969561,200140251,100247275,30000002.0,01/05/2017 00:24:29,1493598269,01/05/2017 00:25:11,1493598311,42,1,visitor_line,1,1,NaN,0,1,1,234,22
2,1555123,200194746,100045597,30000002.0,01/05/2017 00:37:28,1493599048,01/05/2017 00:37:28,1493599048,0,1,visitor_line,0,2,01/05/2017 00:37:32,1,2,1,19,33
3,1555126,200194746,100045597,30000002.0,01/05/2017 00:37:47,1493599067,01/05/2017 00:39:04,1493599144,77,1,visitor_line,0,48,NaN,0,3,1,38,33
4,1555129,200194746,100045597,30000002.0,01/05/2017 00:39:50,1493599190,01/05/2017 00:41:11,1493599271,81,1,visitor_line,0,7,NaN,0,4,1,21,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2928829,4448792,200208938,100144721,30001566.0,31/05/2017 22:41:10,1496270470,31/05/2017 22:41:10,1496270470,22,1,visitor_line,0,31,NaN,1,1636666,0,471,22
2928830,1188446,200021764,100034751,30001566.0,31/05/2017 22:49:01,1496270941,31/05/2017 22:49:07,1496270947,6,1,visitor_line,0,1,NaN,0,1636667,0,80,22
2928831,4448792,200208938,100144721,30001566.0,31/05/2017 22:41:10,1496270470,31/05/2017 22:41:10,1496270470,22,1,visitor_line,0,31,NaN,1,1636667,1,557,22
2928832,1188446,200021764,100034751,30001566.0,31/05/2017 22:49:01,1496270941,31/05/2017 22:49:07,1496270947,6,1,visitor_line,0,1,NaN,0,1636668,1,144,22


In [6]:
df_choice_set[df_choice_set['end_date'].isna()].shape

(0, 19)

In [7]:
df_choice_set.drop(index= 131842, inplace= True)

In [8]:
df_choice_set['end_date'] = pd.to_datetime(df_choice_set['end_date'], format= 'mixed')
df_choice_set[df_choice_set['end_date'].isna()].shape

(0, 19)

In [9]:
session_events_merged[' end_date'] = pd.to_datetime(session_events_merged[' end_date'], format= 'mixed')
session_events_merged[session_events_merged[' end_date'].isna()].shape

(0, 33)

In [10]:
df_choice_set = df_choice_set.reset_index(drop = True)

df_choice_set

,event_id,id_visitor,id_session,id_rep,start_date,start_time,end_date,end_time,duration,event_type,event_type_desc,sentiment,number_words,assignment_date,first_msg_dummy,choice_set,chosen,waiting_time,workload
0,5969556,200140251,100247275,30000002.0,01/05/2017 00:22:57,1493598177,2017-01-05 00:22:57,1493598177,0,1,visitor_line,0,36,01/05/2017 00:24:04,1,0,1,81,22
1,5969561,200140251,100247275,30000002.0,01/05/2017 00:24:29,1493598269,2017-01-05 00:25:11,1493598311,42,1,visitor_line,1,1,NaN,0,1,1,234,22
2,1555123,200194746,100045597,30000002.0,01/05/2017 00:37:28,1493599048,2017-01-05 00:37:28,1493599048,0,1,visitor_line,0,2,01/05/2017 00:37:32,1,2,1,19,33
3,1555126,200194746,100045597,30000002.0,01/05/2017 00:37:47,1493599067,2017-01-05 00:39:04,1493599144,77,1,visitor_line,0,48,NaN,0,3,1,38,33
4,1555129,200194746,100045597,30000002.0,01/05/2017 00:39:50,1493599190,2017-01-05 00:41:11,1493599271,81,1,visitor_line,0,7,NaN,0,4,1,21,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2928828,4448792,200208938,100144721,30001566.0,31/05/2017 22:41:10,1496270470,2017-05-31 22:41:10,1496270470,22,1,visitor_line,0,31,NaN,1,1636666,0,471,22
2928829,1188446,200021764,100034751,30001566.0,31/05/2017 22:49:01,1496270941,2017-05-31 22:49:07,1496270947,6,1,visitor_line,0,1,NaN,0,1636667,0,80,22
2928830,4448792,200208938,100144721,30001566.0,31/05/2017 22:41:10,1496270470,2017-05-31 22:41:10,1496270470,22,1,visitor_line,0,31,NaN,1,1636667,1,557,22
2928831,1188446,200021764,100034751,30001566.0,31/05/2017 22:49:01,1496270941,2017-05-31 22:49:07,1496270947,6,1,visitor_line,0,1,NaN,0,1636668,1,144,22


In [11]:
df_choice_set['end_date'].dtype

dtype('<M8[ns]')

In [12]:
df_choice_set['waiting_time'] = df_choice_set['waiting_time'].astype(int)

In [13]:
df_choice_set['chosen_time'] = df_choice_set['end_date'] + pd.to_timedelta(df_choice_set['waiting_time'], unit='s')

In [14]:
df_choice_set = df_choice_set[['id_session', 'id_rep', 'end_date', 'waiting_time', 'chosen_time', 'choice_set', 'event_id']].sort_values(by = (['id_rep', 'end_date']))

In [15]:
session_events_merged.rename(columns= {' id_session': 'id_session', ' event_type_desc': 'event_type_desc', ' end_date': 'end_date', ' id_rep': 'id_rep', ' end_time': 'end_time', ' event_type': 'event_type'}, inplace = True)

In [24]:
session_events_merged = session_events_merged[['id_session', 'event_type_desc', 'end_date', 'id_rep', 'event_id', 'event_type']].sort_values(by = ['id_rep', 'end_date'])

In [17]:
session_events_merged = session_events_merged[~((session_events_merged['event_type_desc'] == ' other_time') | (session_events_merged['event_type_desc'] == ' inner_wait'))].reset_index(drop = True)

In [18]:
session_events_merged.dtypes

id_session                  int64
event_type_desc            object
end_date           datetime64[ns]
id_rep                    float64
end_time                    int64
event_id                    int64
event_type                  int64
dtype: object

In [19]:
df_choice_set.dtypes

id_session              object
id_rep                  object
end_date        datetime64[ns]
waiting_time             int64
chosen_time     datetime64[ns]
choice_set              object
event_id                object
dtype: object

In [20]:
df_choice_set['id_session'] = df_choice_set['id_session'].astype(int)

---------------------

In [22]:
df_choice_set.head()

,id_session,id_rep,end_date,waiting_time,chosen_time,choice_set,event_id
0,100247275,30000002.0,2017-01-05 00:22:57,81,2017-01-05 00:24:18,0,5969556
1,100247275,30000002.0,2017-01-05 00:25:11,234,2017-01-05 00:29:05,1,5969561
2,100045597,30000002.0,2017-01-05 00:37:28,19,2017-01-05 00:37:47,2,1555123
3,100045597,30000002.0,2017-01-05 00:39:04,38,2017-01-05 00:39:42,3,1555126
4,100045597,30000002.0,2017-01-05 00:41:11,21,2017-01-05 00:41:32,4,1555129


In [29]:
session_events_merged_sliced.head(40)

,id_session,event_type_desc,end_date,id_rep,event_id,event_type
0,100115501,visitor_line,2017-01-05 00:06:20,30000002.0,3708824,1
1,100115501,rep_line,2017-01-05 00:06:31,30000002.0,3708826,2
2,100115501,visitor_line,2017-01-05 00:06:39,30000002.0,3708827,1
3,100115501,rep_line,2017-01-05 00:07:11,30000002.0,3708828,2
4,100115501,rep_line,2017-01-05 00:07:15,30000002.0,3708829,2
5,100115501,visitor_line,2017-01-05 00:07:21,30000002.0,3708830,1
6,100115501,rep_line,2017-01-05 00:11:28,30000002.0,3708831,2
7,100115501,visitor_line,2017-01-05 00:12:09,30000002.0,3708832,1
8,100115501,rep_line,2017-01-05 00:14:54,30000002.0,3708833,2
9,100115501,rep_line,2017-01-05 00:15:15,30000002.0,3708834,2


In [33]:
session_events_merged_sliced.iloc[490:510]

,id_session,event_type_desc,end_date,id_rep,event_id,event_type
490,100014492,rep_line,2017-01-05 07:16:13,30000002.0,494729,2
491,100000141,visitor_line,2017-01-05 07:16:22,30000002.0,4759,1
492,100000141,rep_line,2017-01-05 07:16:38,30000002.0,4760,2
493,100014492,rep_line,2017-01-05 07:17:09,30000002.0,494731,2
494,100030938,visitor_line,2017-01-05 07:20:20,30000002.0,1057870,1
495,100014492,visitor_line,2017-01-05 07:20:33,30000002.0,494733,1
496,100030938,rep_line,2017-01-05 07:20:43,30000002.0,1057873,2
497,100030938,rep_line,2017-01-05 07:21:05,30000002.0,1057874,2
498,100014492,rep_line,2017-01-05 07:22:35,30000002.0,494735,2
499,100014492,rep_line,2017-01-05 07:23:20,30000002.0,494736,2


-----------------------

In [58]:
df_choice_set_sliced = df_choice_set.iloc[:1000].reset_index(drop = True)
session_events_merged_sliced = session_events_merged.iloc[:5000].reset_index(drop = True)

In [45]:
import pandas as pd
import numpy as np

# --- Assumptions: your frames already exist with these columns ---
# df_choice_set: columns ['id_session', 'chosen_time']
# merged_session_events: columns ['id_session', 'end_date']

# (Optional) Trim to needed columns and dtypes to save memory for large data
df_choice_set_sliced = df_choice_set_sliced[['id_session', 'chosen_time']]
session_events_merged_sliced = session_events_merged_sliced[['id_session', 'end_date', *[c for c in session_events_merged_sliced.columns if c not in ('id_session', 'end_date')]]]

# Sort for group ops and asof
df_choice_set_sliced = df_choice_set_sliced.sort_values(['id_session', 'chosen_time'], kind='mergesort')
session_events_merged_sliced = session_events_merged_sliced.sort_values(['id_session', 'end_date'], kind='mergesort')

# Build windows per session: start = chosen_time, end = lead(chosen_time)
df_choice_set_sliced['window_end'] = df_choice_set_sliced.groupby('id_session')['chosen_time'].shift(-1)

# Drop the last choice per session (no closing end)
choice_windows = df_choice_set_sliced.dropna(subset=['window_end']).copy()

# Stable window id (unique across all sessions)
# Option A: sequential per session then combine
choice_windows['window_seq'] = choice_windows.groupby('id_session').cumcount()
choice_windows['choice_window_id'] = (
    choice_windows['id_session'].astype(str) + '_' + choice_windows['window_seq'].astype(str)
)

# Keep only columns needed for the join
choice_windows = choice_windows[['id_session', 'chosen_time', 'window_end', 'choice_window_id']]
choice_windows = choice_windows.rename(columns={'chosen_time': 'window_start'})

# Sort before merge
session_events_merged_sliced = session_events_merged_sliced.sort_values('end_date').reset_index(drop=True)
choice_windows = choice_windows.sort_values('window_start').reset_index(drop=True)


# As-of merge: for each event, find the nearest window_start at or before end_date, by session
# Requirements: both frames sorted by 'end_date'/'window_start' within 'id_session'
tagged = pd.merge_asof(
    session_events_merged_sliced,
    choice_windows,
    left_on='end_date',
    right_on='window_start',
    by='id_session',
    direction='backward',
    allow_exact_matches=True
)

# Keep only events that actually fall inside the window: end_date in [window_start, window_end)
mask_in_window = (tagged['window_start'].notna()) & (tagged['end_date'] < tagged['window_end'])
tagged_in_window = tagged.loc[mask_in_window].copy()

# Result:
# - 'tagged_in_window' contains only rows within a window, with 'choice_window_id' attached.
# - If you want the full events table with NA for events outside windows, use `tagged` instead.


In [75]:
session_events_merged_sliced.shape

(5000, 6)

In [77]:
df_choice_set_sliced.shape

(1000, 9)

(1000, 9)
(1000, 9)
(5000, 10)
(5000, 9)
(5000,)
(11, 10)


In [67]:
session_events_merged_sliced['id_rep'].dtype

dtype('float64')

In [93]:
print(session_events_merged_sliced['id_rep'].dtype)
print(df_choice_set_sliced['id_rep'].dtype)


float64
float64


In [68]:
df_choice_set_sliced['id_rep'] = df_choice_set_sliced['id_rep'].astype('float64')

In [92]:
hits

,id_rep,id_session,msg_event_id,end_date,event_type,event_type_desc,turn_id,window_end,window_start,window_type
0,30000002.0,100115501,3708824,2017-01-05 00:06:20,1,visitor_line,5969556,2017-01-05 00:24:18,2017-01-05 00:06:20,not_chosen
1,30000002.0,100115501,3708826,2017-01-05 00:06:31,2,rep_line,5969556,2017-01-05 00:24:18,2017-01-05 00:06:20,not_chosen
2,30000002.0,100115501,3708827,2017-01-05 00:06:39,1,visitor_line,5969556,2017-01-05 00:24:18,2017-01-05 00:06:20,not_chosen
3,30000002.0,100115501,3708828,2017-01-05 00:07:11,2,rep_line,5969556,2017-01-05 00:24:18,2017-01-05 00:06:20,not_chosen
4,30000002.0,100115501,3708829,2017-01-05 00:07:15,2,rep_line,5969556,2017-01-05 00:24:18,2017-01-05 00:06:20,not_chosen
5,30000002.0,100115501,3708830,2017-01-05 00:07:21,1,visitor_line,5969556,2017-01-05 00:24:18,2017-01-05 00:06:20,not_chosen
6,30000002.0,100115501,3708831,2017-01-05 00:11:28,2,rep_line,5969556,2017-01-05 00:24:18,2017-01-05 00:06:20,not_chosen
7,30000002.0,100115501,3708832,2017-01-05 00:12:09,1,visitor_line,5969556,2017-01-05 00:24:18,2017-01-05 00:06:20,not_chosen
8,30000002.0,100115501,3708833,2017-01-05 00:14:54,2,rep_line,5969556,2017-01-05 00:24:18,2017-01-05 00:06:20,not_chosen
9,30000002.0,100115501,3708834,2017-01-05 00:15:15,2,rep_line,5969556,2017-01-05 00:24:18,2017-01-05 00:06:20,not_chosen


In [100]:
import pandas as pd

# Load your data (assuming you've loaded it already)
# df_choice_set_sliced = ...
# session_events_merged_sliced = ...

# Ensure time columns are datetime objects
df_choice_set_sliced['chosen_time'] = pd.to_datetime(df_choice_set_sliced['chosen_time'])
session_events_merged_sliced['end_date'] = pd.to_datetime(session_events_merged_sliced['end_date'])

# Sort the choice set by agent and time
df_choice_set_sliced = df_choice_set_sliced.sort_values(['id_rep', 'chosen_time']).reset_index(drop=True)

# Group by agent (id_rep) and find the previous chosen_time for any session
df_choice_set_sliced['last_choice_any'] = df_choice_set_sliced.groupby('id_rep')['chosen_time'].shift(1)

# The very first choice event for an agent will have NaN; we'll handle this later,
# perhaps by using a very early date (e.g., 1900-01-01) or by excluding it if
# no prior history is possible/relevant.


# Group by agent AND session (id_rep, id_session) and find the previous chosen_time
df_choice_set_sliced['last_choice_specific'] = df_choice_set_sliced.groupby(['id_rep', 'id_session'])['chosen_time'].shift(1)

df_choice_set_sliced.rename(columns={'chosen_time': 'window_end'}, inplace=True)


all_relevant_messages = []

# Pre-filter raw data to only include customer messages, as agent responses are the choice events
# and the agent is deciding which customer message to respond to.
customer_messages = session_events_merged_sliced[session_events_merged_sliced['event_type'] == 'customer']

for index, choice_row in df_choice_set_sliced.iterrows():
    agent_id = choice_row['id_rep']
    chosen_session = choice_row['id_session']
    window_end = choice_row['window_end']
    
    # 1. Messages for the CHOSEN session
    # Window: [last_choice_specific, window_end)
    start_chosen = choice_row['last_choice_specific']
    
    # Filter the raw customer messages:
    chosen_messages = customer_messages[
        (customer_messages['id_rep'] == agent_id) &
        (customer_messages['id_session'] == chosen_session) &
        (customer_messages['end_date'] >= start_chosen) &
        (customer_messages['end_date'] < window_end)
    ].copy()
    
    chosen_messages['is_chosen_session'] = True
    chosen_messages['window_start'] = start_chosen
    chosen_messages['window_end'] = window_end
    chosen_messages['choice_event_id'] = index # Unique ID for this specific choice event

    # 2. Messages for the NON-CHOSEN sessions
    # Window: [last_choice_any, window_end)
    start_non_chosen = choice_row['last_choice_any']
    
    # Filter the raw customer messages:
    # Filter for all sessions EXCEPT the chosen one, within the agent's global window
    non_chosen_messages = customer_messages[
        (customer_messages['id_rep'] == agent_id) &
        (customer_messages['id_session'] != chosen_session) &
        (customer_messages['end_date'] >= start_non_chosen) &
        (customer_messages['end_date'] < window_end)
    ].copy()

    non_chosen_messages['is_chosen_session'] = False
    non_chosen_messages['window_start'] = start_non_chosen
    non_chosen_messages['window_end'] = window_end
    non_chosen_messages['choice_event_id'] = index # Unique ID for this specific choice event

    # Combine and store
    all_relevant_messages.append(chosen_messages)
    all_relevant_messages.append(non_chosen_messages)

# Final DataFrame of all messages mapped to their relevant choice event and window
df_relevant_messages = pd.concat(all_relevant_messages, ignore_index=True)



# Sort by choice event ID, session, and time (important!)
df_relevant_messages.sort_values(
    ['choice_event_id', 'id_session', 'end_date'],
    inplace=True
)

# Calculate the time difference between consecutive messages within the same session
# and the same choice event.
df_relevant_messages['time_diff'] = df_relevant_messages.groupby(
    ['choice_event_id', 'id_session']
)['end_date'].diff()

# Define a maximum gap for a "consecutive" message to be part of the same turn.
# *Crucial Decision:* You need to define this time threshold (e.g., 5 minutes).
MAX_TURN_GAP = pd.Timedelta(minutes=5)

# A new turn starts if:
# 1. It's the first message of a session/choice event group (time_diff is NaN).
# 2. The time difference to the previous message exceeds the MAX_TURN_GAP.
df_relevant_messages['new_turn_flag'] = (
    df_relevant_messages['time_diff'].isna() |
    (df_relevant_messages['time_diff'] > MAX_TURN_GAP)
)

# Create the turn ID by cumulatively summing the 'new_turn_flag' within each group
df_relevant_messages['turn_sequence_id'] = df_relevant_messages.groupby(
    ['choice_event_id', 'id_session']
)['new_turn_flag'].cumsum()

In [102]:
all_relevant_messages

[Empty DataFrame
 Columns: [id_session, event_type_desc, end_date, id_rep, event_id, event_type, is_chosen_session, window_start, window_end, choice_event_id]
 Index: [],
 Empty DataFrame
 Columns: [id_session, event_type_desc, end_date, id_rep, event_id, event_type, is_chosen_session, window_start, window_end, choice_event_id]
 Index: [],
 Empty DataFrame
 Columns: [id_session, event_type_desc, end_date, id_rep, event_id, event_type, is_chosen_session, window_start, window_end, choice_event_id]
 Index: [],
 Empty DataFrame
 Columns: [id_session, event_type_desc, end_date, id_rep, event_id, event_type, is_chosen_session, window_start, window_end, choice_event_id]
 Index: [],
 Empty DataFrame
 Columns: [id_session, event_type_desc, end_date, id_rep, event_id, event_type, is_chosen_session, window_start, window_end, choice_event_id]
 Index: [],
 Empty DataFrame
 Columns: [id_session, event_type_desc, end_date, id_rep, event_id, event_type, is_chosen_session, window_start, window_end, cho

In [49]:
tagged.iloc[50:100].sort_values(by = ['window_start', 'window_end'])

,id_session,end_date,event_type_desc,id_rep,event_id,event_type,window_start,window_end,choice_window_id
61,100247275,2017-01-05 01:04:48,rep_line,30000002.0,5969576,2,2017-01-05 00:58:12,2017-01-05 00:59:13,100247275_3
50,100045597,2017-01-05 00:59:56,rep_line,30000002.0,1555144,2,2017-01-05 00:59:56,2017-01-05 01:01:57,100045597_6
51,100045597,2017-01-05 01:00:00,rep_line,30000002.0,1555145,2,2017-01-05 00:59:56,2017-01-05 01:01:57,100045597_6
52,100045597,2017-01-05 01:00:28,rep_line,30000002.0,1555146,2,2017-01-05 00:59:56,2017-01-05 01:01:57,100045597_6
53,100045597,2017-01-05 01:01:25,visitor_line,30000002.0,1555147,1,2017-01-05 00:59:56,2017-01-05 01:01:57,100045597_6
54,100045597,2017-01-05 01:01:57,rep_line,30000002.0,1555148,2,2017-01-05 01:01:57,2017-01-05 01:04:13,100045597_7
55,100045597,2017-01-05 01:02:15,visitor_line,30000002.0,1555149,1,2017-01-05 01:01:57,2017-01-05 01:04:13,100045597_7
56,100045597,2017-01-05 01:02:22,visitor_line,30000002.0,1555150,1,2017-01-05 01:01:57,2017-01-05 01:04:13,100045597_7
58,100045597,2017-01-05 01:04:13,rep_line,30000002.0,1555151,2,2017-01-05 01:04:13,2017-01-05 01:07:23,100045597_8
60,100045597,2017-01-05 01:04:39,rep_line,30000002.0,1555153,2,2017-01-05 01:04:13,2017-01-05 01:07:23,100045597_8


In [40]:
df_choice_set.head(50)

,id_session,id_rep,end_date,waiting_time,chosen_time,choice_set,event_id
0,100247275,30000002.0,2017-01-05 00:22:57,81,2017-01-05 00:24:18,0,5969556
1,100247275,30000002.0,2017-01-05 00:25:11,234,2017-01-05 00:29:05,1,5969561
2,100045597,30000002.0,2017-01-05 00:37:28,19,2017-01-05 00:37:47,2,1555123
3,100045597,30000002.0,2017-01-05 00:39:04,38,2017-01-05 00:39:42,3,1555126
4,100045597,30000002.0,2017-01-05 00:41:11,21,2017-01-05 00:41:32,4,1555129
5,100045597,30000002.0,2017-01-05 00:52:48,195,2017-01-05 00:56:03,5,1555131
6,100045597,30000002.0,2017-01-05 00:56:54,13,2017-01-05 00:57:07,6,1555134
7,100247275,30000002.0,2017-01-05 00:57:18,26,2017-01-05 00:57:44,7,5969569
8,100045597,30000002.0,2017-01-05 00:57:44,28,2017-01-05 00:58:12,8,1555137
9,100247275,30000002.0,2017-01-05 00:57:59,13,2017-01-05 00:58:12,8,5969572


In [ ]:
tagged[tagged['event_type_desc'] == ' visitor_line'].sort_values(by = 'choice_window_id').head(50)

In [ ]:
tagged_column = tagged[tagged['event_type_desc'] == ' visitor_line']['choice_window_id'].value_counts()

In [ ]:
(tagged_column > 1).sum()

In [ ]:
tagged_column.count()

This is the place we stopped after the current merge, we think that the merge_asof worked. We need to do more checks before we apply to all data and present to galit

In addition we need to think of solutions to the first rows that did not get tagged, and to make sure that the same rows appear again according to the correct tags in their choice sets (rows that appear a few times, in different choice sets, got tagged correctly) 25/10

In [ ]:
tagged[tagged['event_type_desc'] == ' visitor_line'].shape

In [ ]:
df_choice_set.head(50)

In [ ]:
out_df = tag_messages_with_choiceset(session_events_merged,df_choice_set)


In [ ]:
out_df.head(100)

In [ ]:
out_df.to_csv('before_calculation_of_new_variables_7_10.csv', index = 0)

In [ ]:
tagged_df = pd.DataFrame(tagged_df[0])
tagged_df.rename(columns={tagged_df.iloc[:,2]: 'end_date_shit'}, inplace = True)

tagged_df

In [ ]:
# tagged_df = pd.DataFrame(tagged_df[0])
tagged_df.columns = ['id_session', 'cs_to_delete', 'end_date', 'cs_end_date']

In [ ]:
#tagged_df.drop(columns= 'cs_to_delete', inplace=  True)

tagged_df['composite_key'] = tagged_df['id_session'].astype(str) + "_" + tagged_df["cs_end_date"].astype(str)

tagged_df["id_for_new_variables"] = tagged_df["composite_key"].astype("category").cat.codes


tagged_df.head(100)

In [ ]:
tagged_df['composite_key'].value_counts()

In [ ]:
out = session_events_merged_sliced.copy()
out = out.merge(tagged_df, on=["id_session", 'end_date'], how="left")
out["inside_choice_set"] = out["cs_end_date"].notna()

In [ ]:
session_events_merged.head(100)

In [ ]:
df_choice_set.sort_values(by = ['id_rep', 'chosen_time']).head(200)

In [ ]:
df_choice_set[df_choice_set['id_session'] == 100006449]

In [ ]:
session_events_merged[session_events_merged['id_session'] == 100006449]

In [12]:
df_temp

,event_id,id_visitor,id_session,id_rep,start_date,start_time,end_date,end_time,duration,event_type,event_type_desc,sentiment,number_words,assignment_date,first_msg_dummy
0,0,200064419,100000002,30001090.0,24/05/2017 22:57:02,1495666622,24/05/2017 22:57:02,1495666622,0,1,visitor_line,0,2,24/05/2017 22:57:04,1
1,3,200064419,100000002,30001090.0,24/05/2017 22:57:19,1495666639,24/05/2017 22:57:31,1495666651,12,2,rep_line,0,60,nan,0
2,5,200064419,100000002,30001090.0,24/05/2017 22:57:53,1495666673,24/05/2017 22:58:30,1495666710,37,1,visitor_line,0,15,nan,0
3,7,200064419,100000002,30001090.0,24/05/2017 22:58:52,1495666732,24/05/2017 22:59:26,1495666766,34,2,rep_line,0,9,nan,0
4,9,200064419,100000002,30001090.0,24/05/2017 23:00:21,1495666821,24/05/2017 23:00:26,1495666826,5,1,visitor_line,0,20,nan,0
5,11,200064419,100000002,30001090.0,24/05/2017 23:01:03,1495666863,24/05/2017 23:01:24,1495666884,21,2,rep_line,0,7,nan,0
6,12,200064419,100000002,30001090.0,24/05/2017 23:01:24,1495666884,24/05/2017 23:01:57,1495666917,33,1,visitor_line,0,4,nan,0
7,14,200064419,100000002,30001090.0,24/05/2017 23:02:21,1495666941,24/05/2017 23:02:33,1495666953,12,2,rep_line,0,6,nan,0
8,15,200064419,100000002,30001090.0,24/05/2017 23:02:33,1495666953,24/05/2017 23:02:49,1495666969,16,1,visitor_line,0,1,nan,0
9,17,200064419,100000002,30001090.0,24/05/2017 23:03:12,1495666992,24/05/2017 23:03:21,1495667001,9,2,rep_line,0,4,nan,0


---------------------

In [15]:
import pandas as pd
import os as os
os.chdir(r'C:\Users\Tamir\OneDrive - Technion\current_work_13_08\service systems')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
df_out = pd.read_csv('df_with_periods.csv')

In [16]:
df_raw = pd.read_csv('raw_session_events_before_events_8_9_19_07.csv')
df_raw['assignment_date'] = df_raw['assignment_date'].astype(str)
df_raw = df_raw[['event_id', " id_visitor", ' id_session', " id_rep", " start_date", " start_time"," end_date",
                       " end_time"," duration", " event_type", " event_type_desc", ' sentiment', ' number_words', 'assignment_date', 'first_msg_dummy']].reset_index(drop=True)
# We could slice here to decrease loading time in the big processing code.
# These variables seems not relevant since they regard to agents handling parallel chats and the time the customer waits in these occasions.
dele = df_raw[(df_raw[' event_type'] == 8) | (df_raw[' event_type'] == 9)].index
df_raw.drop(dele , inplace=True)
# Sorting again the data:
df_raw = df_raw.sort_values(by=[' id_session', ' end_time'])
df_raw =  df_raw.reset_index(drop=True)
df_raw.shape

C:\Users\Tamir\AppData\Local\Temp\ipykernel_29328\3435543428.py:1: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('raw_session_events_before_events_8_9_19_07.csv')


(4964895, 15)

In [17]:
df_raw.rename(columns= {' id_session': 'id_session', ' end_date': 'end_date'}, inplace= True)

In [18]:
df_raw_assignment = df_raw[~(df_raw['assignment_date'] == 'nan')]

df_raw_assignment

,event_id,id_visitor,id_session,id_rep,start_date,start_time,end_date,end_time,duration,event_type,event_type_desc,sentiment,number_words,assignment_date,first_msg_dummy
0,0,200064419,100000002,30001090.0,24/05/2017 22:57:02,1495666622,24/05/2017 22:57:02,1495666622,0,1,visitor_line,0,2,24/05/2017 22:57:04,1
39,57,200012444,100000003,30000174.0,17/05/2017 10:22:42,1495016562,17/05/2017 10:46:44,1495018004,1442,1,visitor_line,0,51,17/05/2017 12:22:16,1
44,64,200196308,100000004,30000531.0,18/05/2017 03:37:29,1495078649,18/05/2017 03:37:41,1495078661,12,1,visitor_line,0,18,18/05/2017 03:59:35,1
62,84,200105404,100000005,30000826.0,05/05/2017 07:00:23,1493967623,05/05/2017 07:00:23,1493967623,0,1,visitor_line,0,68,05/05/2017 07:00:27,1
66,90,200220896,100000006,30000149.0,16/05/2017 23:35:10,1494977710,16/05/2017 23:35:10,1494977710,0,1,visitor_line,0,7,16/05/2017 23:35:17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4941686,6590319,200023965,100343982,30000853.0,13/05/2017 16:02:49,1494691369,13/05/2017 16:02:49,1494691369,0,1,visitor_line,0,8,13/05/2017 16:18:14,1
4941690,6590325,200024022,100343984,30000636.0,20/05/2017 17:48:39,1495302519,20/05/2017 17:48:39,1495302519,0,1,visitor_line,0,5,20/05/2017 18:06:16,1
4941695,6590332,200024030,100343986,30000604.0,13/05/2017 22:04:44,1494713084,13/05/2017 22:04:44,1494713084,0,1,visitor_line,0,6,13/05/2017 22:18:25,1
4941701,6590340,200024059,100343987,30000142.0,14/05/2017 12:16:42,1494764202,14/05/2017 12:16:42,1494764202,0,1,visitor_line,0,6,14/05/2017 12:16:44,1


In [19]:
df_raw_assignment = df_raw_assignment[~df_raw_assignment['id_session'].duplicated()]

In [ ]:
import pandas as pd

# Build a mapping: id_session -> assignment_date
assign_map = df_raw_assignment.set_index("id_session")["assignment_date"]

# Fill missing start_time by matching id_session
df_out_sliced["start_time"] = df_out_sliced["start_time"].fillna(
    df_out_sliced["id_session"].map(assign_map)
)


C:\Users\Tamir\AppData\Local\Temp\ipykernel_26096\3805692434.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_out_sliced["start_time"] = df_out_sliced["start_time"].fillna(


In [ ]:
df_out_sliced_ids = df_out_sliced['id_session'].unique()

df_out_sliced_ids

array([100247275, 100045597, 100057443, 100258851, 100063972, 100151963,
       100070858, 100189251, 100044608, 100132142, 100217273, 100272337,
       100082855, 100115605, 100181148, 100189889, 100014492, 100136051,
       100254987, 100074168, 100126434, 100000141, 100030938, 100026233,
       100062324, 100164314, 100285037, 100112012, 100114766, 100241293,
       100308230, 100062779, 100166775, 100049582, 100029275, 100006449,
       100309267, 100177658, 100157885, 100044730, 100115854, 100339922,
       100093806, 100261952, 100199453, 100325498, 100080639, 100276390,
       100108988, 100096280, 100119629, 100011227, 100025874, 100045042,
       100144461, 100251734, 100188706, 100295384, 100283610, 100138122,
       100267265, 100248876, 100204298, 100177902, 100013802, 100135843,
       100008948, 100003789, 100042254, 100173800, 100030748, 100066140,
       100070696, 100227678, 100181710, 100038743, 100303132, 100144421,
       100096984, 100137674, 100300498, 100177803, 

In [ ]:
df_raw_sliced = df_raw[df_raw['id_session'].isin(df_out_sliced_ids)]

df_raw_sliced

,event_id,id_visitor,id_session,id_rep,start_date,start_time,end_date,end_time,duration,event_type,event_type_desc,sentiment,number_words,assignment_date,first_msg_dummy
3754,4727,200196314,100000141,30000002.0,01/05/2017 06:43:57,1493621037,01/05/2017 06:43:57,1493621037,0,1,visitor_line,0,20,01/05/2017 06:43:59,1
3755,4729,200196314,100000141,30000002.0,01/05/2017 06:43:59,1493621039,01/05/2017 06:44:34,1493621074,35,2,rep_line,0,11,nan,0
3756,4730,200196314,100000141,30000002.0,01/05/2017 06:44:34,1493621074,01/05/2017 06:45:15,1493621115,41,1,visitor_line,0,11,nan,0
3757,4731,200196314,100000141,30000002.0,01/05/2017 06:45:15,1493621115,01/05/2017 06:45:34,1493621134,19,2,rep_line,0,58,nan,0
3758,4733,200196314,100000141,30000002.0,01/05/2017 06:49:40,1493621380,01/05/2017 06:51:52,1493621512,132,1,visitor_line,1,37,nan,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4930365,6572973,200214466,100339922,30000002.0,02/05/2017 03:47:37,1493696857,02/05/2017 03:48:48,1493696928,71,2,rep_line,0,41,nan,0
4930366,6572974,200214466,100339922,30000002.0,02/05/2017 03:48:48,1493696928,02/05/2017 03:49:03,1493696943,15,2,rep_line,0,11,nan,0
4930367,6572975,200214466,100339922,30000002.0,02/05/2017 03:49:03,1493696943,02/05/2017 03:50:11,1493697011,68,2,rep_line,0,36,nan,0
4930368,6572977,200214466,100339922,30000002.0,02/05/2017 03:51:42,1493697102,02/05/2017 03:52:26,1493697146,44,2,rep_line,0,26,nan,0


In [ ]:
import pandas as pd

def tag_messages_long_with_event_id(
    df_intervals: pd.DataFrame,
    df_msgs: pd.DataFrame,
    session_col: str = "id_session",
    interval_start_col: str = "start_time",
    interval_end_col: str = "end_time",
    msg_time_col: str = "end_date",     # message timestamp
    msg_id_col: str = "event_id",       # not necessarily unique
    out_tag_col: str = "tag_id",
    out_msg_row_col: str = "msg_row_id",
    inclusive: str = "both",            # "both", "left", "right", "neither"
    keep_unmatched: bool = False,
) -> pd.DataFrame:
    """
    LONG output: duplicates messages per matching interval tag.

    Match rule per session:
      start_time <= end_date <= end_time (depending on inclusive)

    Assumes start_time has already been filled upstream.
    Uses msg_row_id (row-unique) to avoid duplications when event_id repeats.
    """

    intervals = df_intervals.copy()
    msgs = df_msgs.copy()

    # Create unique message row id (prevents merge multiplication if event_id repeats)
    msgs = msgs.reset_index(drop=False).rename(columns={"index": out_msg_row_col})

    # Create tag_id for intervals from row index if missing
    if out_tag_col not in intervals.columns:
        intervals = intervals.reset_index(drop=False).rename(columns={"index": out_tag_col})

    # Parse datetimes
    intervals[interval_start_col] = pd.to_datetime(intervals[interval_start_col], format= 'mixed')
    intervals[interval_end_col]   = pd.to_datetime(intervals[interval_end_col], format= 'mixed')
    msgs[msg_time_col]            = pd.to_datetime(msgs[msg_time_col], format= 'mixed')

    # Drop unusable rows
    msgs = msgs.dropna(subset=[session_col, msg_time_col, msg_id_col])
    # Now we REQUIRE start_time and end_time to be present
    intervals = intervals.dropna(subset=[session_col, interval_start_col, interval_end_col])

    # Unify session dtype
    intervals[session_col] = intervals[session_col].astype(str)
    msgs[session_col] = msgs[session_col].astype(str)

    # Remove invalid intervals (end < start)
    intervals = intervals[intervals[interval_end_col] >= intervals[interval_start_col]].copy()

    # Inclusive boundary behavior
    def started(start, t):
        return start <= t if inclusive in ("both", "left") else start < t
    def not_ended(end, t):
        return end >= t if inclusive in ("both", "right") else end > t

    # Sort for sweep
    msgs = msgs.sort_values([session_col, msg_time_col])
    intervals = intervals.sort_values([session_col, interval_start_col])

    pairs = []  # (id_session, msg_row_id, tag_id)

    for sid, m in msgs.groupby(session_col, sort=False):
        itv = intervals[intervals[session_col] == sid]
        if itv.empty:
            if keep_unmatched:
                for rid in m[out_msg_row_col].to_numpy():
                    pairs.append((sid, rid, pd.NA))
            continue

        m = m.sort_values(msg_time_col)
        itv = itv.sort_values(interval_start_col)

        itv_tag   = itv[out_tag_col].to_numpy()
        itv_start = itv[interval_start_col].to_numpy()
        itv_end   = itv[interval_end_col].to_numpy()

        active = []  # list of (end_time, tag_id)
        j = 0

        for rid, t in zip(m[out_msg_row_col].to_numpy(), m[msg_time_col].to_numpy()):
            # add intervals that have started
            while j < len(itv_start) and started(itv_start[j], t):
                active.append((itv_end[j], itv_tag[j]))
                j += 1

            # keep intervals that haven't ended
            if active:
                active = [(e, tag) for (e, tag) in active if not_ended(e, t)]

            # emit one row per active interval -> duplicates messages per tag
            if active:
                for e, tag in active:
                    pairs.append((sid, rid, tag))
            elif keep_unmatched:
                pairs.append((sid, rid, pd.NA))

    pairs_df = pd.DataFrame(pairs, columns=[session_col, out_msg_row_col, out_tag_col])

    # Attach message columns + interval columns
    tagged_long = (
        pairs_df
        .merge(msgs, on=[session_col, out_msg_row_col], how="left")
        .merge(intervals, on=[session_col, out_tag_col], how="left", suffixes=("_msg", "_itv"))
    )

    return tagged_long

In [ ]:
tagged = tag_messages_long_with_event_id(
    df_intervals=df_out,
    df_msgs=df_raw,
    session_col="id_session",
    interval_start_col="start_time",
    interval_end_col="end_time",
    msg_time_col="end_date",
    msg_id_col="event_id",
    out_tag_col="tag_id",
    inclusive="both",
    keep_unmatched=False
)

# Example: show duplicated tags for one message
multi = (
    tagged
    .groupby(["id_session", "event_id"])
    .agg(
        n_tags=("tag_id", "nunique"),
        tags=("tag_id", lambda x: sorted(x.unique()))
    )
    .reset_index()
)

multi.query("n_tags > 1").head()